# Práctica 2

Esta práctica ha sido realizada por:  
- **Ana Gil Molina**  
- **Ignacio Ruiz Chicano**  
- **Juan Jesús Torralba Mateos**



## Introducción

En esta práctica se presenta un problema de predicción de series temporales multivaluadas, que consiste en la predicción a 7 días del óxido Nítrico (`NOX`). Se dispone de una serie de datasets entre los años 2017 y 2022, con medidas horarias de una estación sensora situada en la Alojorra. Dichos datasets constan de dos grupos de datos: datos sobre calidad del aire y datos climáticos.

| Tipo de Datos              | Variables              | Unidad de Medida |
|----------------------------|------------------------|------------------|
| **Datos Calidad del Aire** | Óxido Nítrico (NO)     | 𝜇𝑔/𝑚³𝑁         |
|  **Datos Calidad del Aire**                           | Dióxido de Nitrógeno (NO2) | 𝜇𝑔/𝑚³𝑁         |
|   **Datos Calidad del Aire**                          | Disulfuro de Azufre (SO2) | 𝜇𝑔/𝑚³𝑁         |
| **Datos Calidad del Aire** | Ozono (O3)            | 𝜇𝑔/𝑚³𝑁         |
| **Datos Calidad del Aire** | Óxido de Nitrógeno (NOX)            | 𝜇𝑔/𝑚³𝑁         |
|   **Datos Calidad del Aire**                          | PM10                  | 𝜇𝑔/𝑚³𝑁         |
|  **Datos Calidad del Aire**                           | Benceno (C6H6)        | 𝜇𝑔/𝑚³𝑁         |
|   **Datos Calidad del Aire**                          | Tolueno (C7H8)        | 𝜇𝑔/𝑚³𝑁         |
|   **Datos Calidad del Aire**                          | Xileno (XIL)          | 𝜇𝑔/𝑚³𝑁         |
| **Datos Meteorológicos**   | Temperatura media (TMP) | ºC              |
| **Datos Meteorológicos**                           | Humedad Relativa (HR) | % H.R.           |
| **Datos Meteorológicos**                           | Dirección del viento   | º                |
| **Datos Meteorológicos**                           | Presión atmosférica (PRB) | Milibares      |
| **Datos Meteorológicos**                           | Velocidad del Viento (W) | 𝑚/𝑠            |
|  **Datos Meteorológicos**                          | Radiación Solar (RS)   | 𝑊/𝑚³           |

A lo largo de esta práctica, utilizaremos una variedad de modelos, comenzando con enfoques más simples en los que únicamente emplearemos la variable `NOX` para entrenar el modelo y hacer las predicciones. Posteriormente, crearemos modelos más complejos, en los cuales primero incorporaremos los datos de calidad del aire, y seguidamente, usaremos estos mismos modelos, pero añadiendo además los datos meteorológicos para evaluar su impacto en las predicciones.

## Librerias

Para la realización de esta práctica hemos utilizado las siguientes librerías:

In [ ]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import boxcox, shapiro, kurtosis, skew
from scipy.special import inv_boxcox
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import (
    mean_absolute_error, 
    root_mean_squared_error, 
    mean_squared_error, 
    make_scorer, 
    mean_absolute_percentage_error
)

import skforecast
from skforecast.datasets import fetch_dataset
from skforecast.model_selection import backtesting_forecaster, TimeSeriesFold
from skforecast.recursive import ForecasterRecursive, ForecasterEquivalentDate

from IPython.core.magic import register_cell_magic
import IPython

from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import TimeSeriesFold

from skforecast.exceptions import LongTrainingWarning
from skforecast.utils.utils import load_forecaster, save_forecaster

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor


import warnings

import joblib
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.preprocessing import RollingFeatures



In [ ]:
import platform
import importlib.metadata as imp

print(f'Python Version == {platform.python_version()}')

paquetes = [
    'matplotlib',
    'numpy',
    'pandas',
    'seaborn',
    'scikit-learn',
    'scipy',
    'xgboost',
    'skforecast',
    'IPython',
    'statsmodels'
]

for paq in paquetes:
    print(f'{paq} == {imp.version(paq)}')

In [ ]:
warnings.filterwarnings("ignore", message="X does not have valid feature names")
warnings.simplefilter('ignore', category=LongTrainingWarning)

In [ ]:
# Establecemos una semilla
semilla = 123
np.random.seed(semilla)

## Modo de ejecución del notebook

Dependiendo del modo de ejecución del notebook, se ejecutarán todas las celdas o todas las celdas excepto las que contienen la creación de los modelos.
Por defecto, la variable `saltarse_generacion_modelos` está establecida en `True`, por lo que se omitirá la generación de los modelos.

In [ ]:
saltarse_generacion_modelos = False

Esta es la función que nos ayudará a poder ejecutar el notebook en modo de generación de modelos o en modo de carga de modelos.

In [ ]:
@register_cell_magic
def skip_if(line, cell):
    # Evalúa la condición en 'line' y decide si ejecutar la celda
    if eval(line):
        return  # Si la condición es True, no ejecuta la celda
    # Si la condición es False, ejecuta la celda
    get_ipython().run_cell(cell)

## Cargamos los datos

Vamos a leer todos los archivos `.xls` por separado y los vamos a concatenar en un único DataFrame. Además, eliminamos las unidades de medida que acompañan al nombre de las columnas para una mejor legibilidad.

In [ ]:
rutaBase = './data/'
nombreComun = 'Aljorrahorarias'
anos = ['2017', '2018', '2019', '2020', '2021', '2022']

dataframes = []

for ano in anos:
    archivo = f"{rutaBase}{nombreComun}{ano}.xls"
    if os.path.exists(archivo):
        df = pd.read_excel(archivo, engine="xlrd", header=1)
        dataframes.append(df)

df_concatenado = pd.concat(dataframes, ignore_index=True)

df_concatenado.columns = df.columns.str.replace(r"\s*\(.*?\)", "", regex=True)
df_concatenado.to_csv(f"{rutaBase}{nombreComun}2017_2022.csv", index=False)

#df_concatenado_aire = df_concatenado.drop(columns=['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
#df_concatenado_aire.to_csv(f"{rutaBase}{nombreComun}2017_2022_aire.csv", index=False)

Vamos a visualizar las columnas para ver si hemos eliminado bien las unidades de medida.

In [ ]:
df_concatenado.columns

## Análisis exploratorio de los datos

In [ ]:
print(df_concatenado.info())

Tenemos 15 variables numéricas donde tenemos datos procedentes de dos fuentes distintas: La primera fuente es de la calidad del aire y la segunda son datos meteorológicos.

Lo primero que haremos será pasar los meses en español a inglés, para poder trabajar con ellos de forma más sencilla.

In [ ]:
meses_espanol_a_ingles = {
    'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun',
    'jul': 'Jul', 'ago': 'Aug', 'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'
}

### Preprocesamiento de las fechas

Convertimos las fechas a minúsculas y reemplazamos los nombres de los meses en español por sus equivalentes en inglés. Luego, transformamos ambos dataframes al formato `datetime` y configuramos la columna de fecha como índice.

In [ ]:
df_concatenado['Fecha'] = df_concatenado['Fecha'].str.lower().replace(meses_espanol_a_ingles, regex=True)
df_concatenado['Fecha'] = pd.to_datetime(df_concatenado['Fecha'], format='%d %b. %Y %H:%M', errors='coerce')
df_concatenado.set_index('Fecha', inplace=True)

### Estudio de los valores nulos

Queremos conocer la cantidad de valores nulos por columna. Para ello, basta con inspeccionar el dataframe `df_concatenado`, ya que `df_concatenado_aire` es un subconjunto de este.


In [ ]:
nulos = df_concatenado.isnull().sum()
no_nulos = df_concatenado.notnull().sum()

plt.figure(figsize=(12, 6))
plt.bar(df_concatenado.columns, no_nulos, label='Valores No Nulos', color='#98FB98', edgecolor='black')
plt.bar(df_concatenado.columns, nulos, bottom=no_nulos, label='Valores Nulos', color='#FFA07A', edgecolor='black')

plt.title('Valores Nulos y No Nulos por Columna', fontsize=16)
plt.xlabel('Columnas', fontsize=12)
plt.ylabel('Cantidad de valores', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(loc = 'lower left')

plt.tight_layout()
plt.show()

Como podemos observar todas las columnas tienen valores nulos, sin embargo, destaca una de ellas respecto a todas las demás, la columna `Ruido`, la cuál tiene todos los valores `NaN`. Por ello procedemos a eliminarla.

In [ ]:
df_concatenado.drop(columns=['Ruido'], inplace=True)

Aún seguimos teniendo 3 variables que destacan respecto a las demás en cuanto a valores nulos, estas son: `C6H6`, `C7H8` y `XIL`. Vamos a ver cuántos valores con valor `NaN` poseen de manera numérica.

In [ ]:
columna_xil = 'XIL'
columna_c7h8 = 'C7H8'
columna_c6h6 = 'C6H6'

valores_nulos_penultima = df_concatenado[columna_xil].isnull().sum()
valores_nulos_antepenultima = df_concatenado[columna_c7h8].isnull().sum()
valores_nulos_anteantepenultima = df_concatenado[columna_c6h6].isnull().sum()

print(f"Valores nulos de la penúltima columna ({columna_xil}):")
print(valores_nulos_penultima)
print("\n")

print(f"Valores nulos de la antepenúltima columna ({columna_c7h8}):")
print(valores_nulos_antepenultima)
print("\n")

print(f"Valores nulos de la anteantepenúltima columna ({columna_c6h6}):")
print(valores_nulos_anteantepenultima)



Observamos que las tres columnas tienen exactamente la misma cantidad de valores nulos. Ahora, queremos verificar si estos valores nulos se distribuyen de manera similar a lo largo del tiempo.

In [ ]:
null_penultima = df_concatenado[columna_xil].isnull()
null_antepenultima = df_concatenado[columna_c7h8].isnull()
null_anteantepenultima = df_concatenado[columna_c6h6].isnull()

counts_penultima = df_concatenado[null_penultima].resample('ME').size()
counts_antepenultima = df_concatenado[null_antepenultima].resample('ME').size()
counts_anteantepenultima = df_concatenado[null_anteantepenultima].resample('ME').size()

plt.figure(figsize=(14, 8))

plt.subplot(3, 1, 1)
counts_penultima.plot(color='blue', label=columna_xil)
plt.title(f"Distribución de valores nulos en la columna {columna_xil}")
plt.xlabel("Fecha")
plt.ylabel("Cantidad de valores nulos")
plt.legend()

plt.subplot(3, 1, 2)
counts_antepenultima.plot(color='green', label=columna_c7h8)
plt.title(f"Distribución de valores nulos en la columna {columna_c7h8}")
plt.xlabel("Fecha")
plt.ylabel("Cantidad de valores nulos")
plt.legend()

plt.subplot(3, 1, 3)
counts_anteantepenultima.plot(color='red', label=columna_c6h6)
plt.title(f"Distribución de valores nulos en la columna {columna_c6h6}")
plt.xlabel("Fecha")
plt.ylabel("Cantidad de valores nulos")
plt.legend()

plt.tight_layout()
plt.show()

Aunque parece que siguen la misma distribución a lo largo del tiempo, para visualizarlo con mayor claridad, vamos a superponer los valores nulos de las tres columnas en un solo gráfico.


In [ ]:
plt.figure(figsize=(14, 8))

counts_penultima.plot(color='blue', label=columna_xil)
counts_antepenultima.plot(color='green', label=columna_c7h8)
counts_anteantepenultima.plot(color='red', label=columna_c6h6)

plt.title("Distribución superpuesta de valores no nulos en las columnas")
plt.xlabel("Fecha")
plt.ylabel("Cantidad de valores no nulos")
plt.legend()  

plt.tight_layout()
plt.show()

Como se puede observar, las tres variables tienen los valores nulos distribuidos de la misma manera. Dada la alta cantidad de valores nulos en estas columnas, decidimos eliminarlas de los datasets.


In [ ]:
df_concatenado.drop(columns=[columna_c6h6, columna_c7h8, columna_xil], inplace=True)

print("Columnas de df_concatenado:")
print(df_concatenado.columns)

### Estudio de la correlación

Tras decidir qué variables eliminar debido a la gran cantidad de valores nulos, procederemos a analizar la correlación entre las variables restantes. Para ello, utilizaremos un mapa de calor.


In [ ]:
correlation_matrix = df_concatenado.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', cbar=True, square=True, mask=mask)
plt.title("Matriz de Correlación - Parte Inferior")
plt.tight_layout()
plt.show()

Lo más destacable del mapa de calor es que las variable `NOX` tiene una correlación muy alta con las variables `NO` y `NO2`, lo cual es lógico ya que el `NOX` (Óxidos de Nitrógeno) es la suma de `NO` (Óxido Nítrico) y `NO2` (Dióxido de Nitrógeno).

### Análisis a través de boxplots

#### Todas las variables

Vamos a observar a través de gráficas de boxplot la distribución de los valores de las variables predictoras.

In [ ]:
plt.figure(figsize=(15, 10))
sns.set_palette('pastel')
sns.boxplot(data=df_concatenado.select_dtypes(include='number'))

plt.xlabel('Variables numéricas')
plt.ylabel('Distribución')
plt.title('Boxplot de todas las variables numéricas')
plt.xticks(rotation=45) 
plt.show()

Vemos diferentes rangos de valores en las variables predictoras y muchos outliers que pueden ser debidos a los fallos de los sensores.

#### Solo la variable NOX

Realmente el gráfico anterior no nos dice mucho de la variable `NOX`, por lo que vamos a realizar un boxplot de la variable `NOX` a lo largo de diferentes periodos temporales.

Para ello lo primero que vamos a hacer será crear una copia temporal del dataframe `df_concatenado` para poder trabajar con ella sin modificar el dataframe original.

In [ ]:
df_temp = df_concatenado.copy()

Vamos a visualizar la distribución de la variable `NOX` por las horas del día.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
df_temp['Hora'] = df_temp.index.hour + 1
df_temp.boxplot(column='NOX', by='Hora', ax=ax, flierprops={'markersize': 3, 'alpha': 0.3})
df_temp.groupby('Hora')['NOX'].median().plot(style='o-', linewidth=0.8, ax=ax)
ax.set_ylabel('NOX')
ax.set_title('Distribución de NOX por hora del día', fontsize=9)
plt.suptitle("")
plt.show()

Las concentraciones de NOX aumentan de forma significativa entre las **8:00 y las 12:00 horas**, coincidiendo con el inicio y desarrollo de las jornadas laborales. Otro pico notable se registra entre las **20:00 y 24:00 horas**, lo que podría estar relacionado con turnos nocturnos o procesos industriales que continúan operando más allá del horario comercial habitual. En contraste, durante la **madrugada (1:00 a 6:00 horas)**, las concentraciones de NOX son considerablemente más bajas, algo esperable debido a la **reducción de la actividad humana e industrial**, así como del transporte.


Vamos a visualizar la distribución de la variable `NOX` por días de la semana.	

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
df_temp['dia_semana'] = df_temp.index.day_of_week + 1
df_temp.boxplot(column='NOX', by='dia_semana', ax=ax, flierprops={'markersize': 3, 'alpha': 0.3})
df_temp.groupby('dia_semana')['NOX'].median().plot(style='o-', linewidth=0.8, ax=ax)
ax.set_ylabel('NOX')
ax.set_title('Distribución de NOX por día de la semana', fontsize=9)
plt.suptitle("")  # Para eliminar el título adicional
plt.show()

Las concentraciones de NOX se mantienen relativamente **constantes a lo largo de la semana**, con ligeras variaciones y una mayor dispersión de valores atípicos en los días **laborales (lunes a viernes)**, lo que podría estar asociado a la **actividad industrial y el tráfico**. Durante el **fin de semana (sábado y domingo)**, las concentraciones tienden a ser ligeramente menores, reflejando una posible **reducción de la actividad industrial y humana**.


Vamos a visualizar la distribución de la variable `NOX` por meses.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
df_temp['mes'] = df_temp.index.month
df_temp.boxplot(column='NOX', by='mes', ax=ax, flierprops={'markersize': 3, 'alpha': 0.3})
df_temp.groupby('mes')['NOX'].median().plot(style='o-', linewidth=0.8, ax=ax)
ax.set_ylabel('NOX')
ax.set_title('Distribución de NOX por mes', fontsize=9)
plt.suptitle("")  # Para eliminar el título adicional
plt.show()

Las concentraciones de NOX son **más altas en los primeros meses del año (enero a marzo)**, con una mayor dispersión de valores atípicos, lo que sugiere una posible **intensificación de la actividad industrial y de transporte** en esta época. A partir de **abril**, las concentraciones disminuyen progresivamente, alcanzando los niveles más bajos en los meses de **verano (junio a septiembre)**, probablemente debido a una **reducción en la actividad industrial** y mejores condiciones atmosféricas para la dispersión de contaminantes. Hacia el final del año (**noviembre y diciembre**), se observa un ligero repunte, posiblemente relacionado con un aumento en la **actividad comercial e industrial**.


Vamos a visualizar la distribución de la variable `NOX` por meses a lo largo de los años.

In [ ]:
df_temp['YearMonth'] = df_temp.index.to_period('M')  
fig, ax = plt.subplots(figsize=(20, 8))
sns.set_palette('pastel')

sns.boxplot(x='YearMonth', y='NOX', data=df_temp, ax=ax)

ax.set_xlabel('Mes y Año')
ax.set_ylabel('NOX')
ax.set_title('Distribución de NOX por mes a lo largo de los años')
plt.xticks(rotation=45)
plt.show()

Se evidencia una **tendencia bajista** desde 2017. Los valores más altos y dispersos al inicio disminuyen notablemente después de **2020**, coincidiendo posiblemente con la pandemia de COVID-19. Se observan **valores atípicos frecuentes**, especialmente en los primeros meses de cada año (enero y febrero), lo que sugiere patrones estacionales asociados a condiciones climáticas o emisiones. Es importante señalar que **en abril de 2017 no hay datos disponibles de NOX**, lo cual puede influir en el análisis de esa parte del año. Desde **2021**, las concentraciones son más bajas y estables, con menor dispersión y menos valores extremos. En general, la gráfica refleja una **reducción sostenida** de NOX, con picos ocasionales que resaltan en ciertos periodos.


### Interpolación

Vamos a probar la interpolación lineal y observaremos la varianza de la columna NOX

In [ ]:
df_concatenado_interpolado_lineal = df_concatenado.interpolate(method='linear')

# Configuración de gráficos
_, axs = plt.subplots(1, 2, figsize=(20, 10))
series = [
    df_concatenado['NOX'],
    df_concatenado_interpolado_lineal['NOX']
]
titulos = [
    'Distribución Original de NOX',
    'Distribución con Interpolación Lineal'
]

sns.set(font_scale=1)
axs = axs.flatten()

for data, ax, titulo in zip(series, axs, titulos):
    sns.histplot(data, kde=True, stat='count', ax=ax)
    ax.set_title(titulo)
    ax.set_xlabel('Valores de NOX')
    legend = ax.get_legend()
    if legend is not None:
        legend.set_visible(False)

plt.tight_layout()
plt.show()

Podemos notar que la varianza apenas ha sufrido cambios significativos. Ahora, procederemos a verificar si alguna de las columnas contiene valores nulos después de realizar la interpolación.

In [ ]:
print(df_concatenado_interpolado_lineal.isna().sum())

Como lo que nos interesa es predecir la variable `NOX` a siete días, vamos a submuestrear los datos para pasar de observaciones de cada hora a observaciones de cada día. Para ello vamos a usar distintas agregaciones dependiendo de la naturaleza de la variable.


<u>*Variables con Promedio Diario (Media)*</u>

- **Óxido Nítrico (NO)**
- **Dióxido de Nitrógeno (NO2)**
- **Disulfuro de Azufre (SO2)**
- **Óxidos de Nitrógeno (NOX)**
- **Partículas PM10**
- **Temperatura (TMP)**
- **Humedad Relativa (HR)**
- **Presión atmosférica (PRB)**
- **Velocidad del viento (VV)**

Para estos contaminantes y variables meteorológicas, el promedio diario es el método más adecuado para obtener una visión estable y representativa de su comportamiento a lo largo del día. Estas variables tienden a tener variaciones a corto plazo, pero para la predicción de la calidad del aire a 7 días, es importante obtener una visión general de los niveles diarios. El uso del promedio permite capturar las tendencias generales sin que los picos momentáneos alteren excesivamente el análisis.

---

<u>*Variables con Máximo Diario (Máximo)*</u>

- **Ozono (O3)**

El ozono puede presentar picos significativos durante ciertos períodos del día, particularmente en condiciones de alta radiación solar. El **máximo diario** es el método adecuado para capturar estos picos y entender los momentos de mayor concentración.

---

<u>*Variables con Suma Diaria (Suma)*</u>

- **Radiación Solar (RS)**

La radiación solar se acumula durante el día, lo que hace que la **suma diaria** sea el enfoque más adecuado para representar la cantidad total de radiación recibida en un día. Esto es crucial para entender la influencia de la radiación en la generación de ozono (O3) y otros contaminantes fotoquímicos.

---

<u>*Variables con Promedio Vectorial*</u>

- **Dirección del Viento (DD)**

 
La dirección del viento se representa mediante un ángulo, lo que hace que no sea adecuado usar un promedio aritmético simple. En lugar de calcular la media, se realiza un **promedio vectorial** de los componentes del viento (U y V). Este enfoque tiene en cuenta tanto la magnitud como la dirección del viento, lo que permite obtener un valor coherente para la dirección promedio del viento en un día.


Para la agregación por promedio vectorial vamos a usar la siguiente función:

In [ ]:
def promedio_vectorial(direcciones):
    
    # Convertir de grados a radianes
    radianes = np.radians(direcciones)
    
    # Calcular componentes cartesianas
    x = np.cos(radianes)
    y = np.sin(radianes)
    
    # Promediar las componentes x e y
    x_promedio = np.mean(x)
    y_promedio = np.mean(y)
    
    # Calcular el ángulo promedio (dirección del viento)
    direccion_promedio = np.degrees(np.arctan2(y_promedio, x_promedio)) % 360
    return direccion_promedio

Apliquemos la función `promedio_vectorial` a la columna `DD` para obtener la dirección del viento promedio diario y guardémosla en una nueva columna llamada `dd_resample`.

In [ ]:
dd_resample = df_concatenado_interpolado_lineal['DD'].resample('D').apply(promedio_vectorial)
print(dd_resample.head(10))

Defimos el resto de agregaciones para las variables restantes y al final añadimos la columna `dd_resample` al dataframe.

In [ ]:
columnas_media = ['NO', 'NO2', 'SO2', 'NOX', 'PM10', 'TMP', 'HR', 'PRB', 'VV']
columnas_max = ['O3']
columnas_suma = ['RS']

agg_methods = {col: 'mean' for col in columnas_media}
agg_methods.update({col: 'max' for col in columnas_max})
agg_methods.update({col: 'sum' for col in columnas_suma})

df_concatenado_diario = df_concatenado_interpolado_lineal.resample('D').agg(agg_methods)
df_concatenado_diario['DD'] = dd_resample

In [ ]:
print(df_concatenado_diario.head(10))

### Descomposición de la serie temporal NOX

Antes de descomponer la serie en distintos componentes vamos a visualizarla.

In [ ]:
zoom = ('2017-01-01', '2017-06-30')  

fig = plt.figure(figsize=(12, 8))
grid = plt.GridSpec(nrows=8, ncols=1, hspace=0.2, wspace=0)

main_ax = fig.add_subplot(grid[1:3, :])
zoom_ax = fig.add_subplot(grid[4:, :])

df_concatenado_diario['NOX'].plot(ax=main_ax, c='black', alpha=0.5, linewidth=0.5)

min_y = df_concatenado_diario['NOX'].min()
max_y = df_concatenado_diario['NOX'].max()

main_ax.fill_between(zoom, min_y, max_y, facecolor='blue', alpha=0.5, zorder=0)

start_date = df_concatenado_diario.index.min().strftime('%Y-%m-%d')
end_date = df_concatenado_diario.index.max().strftime('%Y-%m-%d')

start_zoom = zoom[0]
start_zoom = pd.to_datetime(start_zoom).strftime('%Y-%m-%d')
end_zoom = zoom[1]
end_zoom = pd.to_datetime(end_zoom).strftime('%Y-%m-%d')


main_ax.set_xlabel('')
main_ax.set_title(f'Concentración de NOX: Desde {start_date} hasta {end_date}', fontsize=10)

df_concatenado_diario.loc[zoom[0]: zoom[1], 'NOX'].plot(ax=zoom_ax, color='blue', linewidth=1)

zoom_ax.set_title(f'Concentración de NOX: Desde {start_zoom} hasta {end_zoom}', fontsize=10)
zoom_ax.set_xlabel('')

plt.subplots_adjust(hspace=0.2)


Como se puede observar, entre mediados de abril y mediados de junio de 2017, aparece una línea diagonal que corresponde a la interpolación lineal aplicada para rellenar los valores nulos. Dado que esta línea no sigue la tendencia de los datos reales, se procederá a eliminar los registros comprendidos entre el 1 de enero de 2017 y el 30 de junio de 2017.

In [ ]:
start_date = '2017-01-01'
end_date = '2017-06-30'

df_concatenado_diario = df_concatenado_diario.loc[~((df_concatenado_diario.index >= start_date) & (df_concatenado_diario.index <= end_date))]

Vamos a comprobar si se ha eliminado correctamente.

In [ ]:
df_concatenado_diario.head()

Como podemos observar, se han eliminado correctamente los registros comprendidos entre el 1 de enero de 2017 y el 30 de junio de 2017. 

Ahora, vamos a descomponer la serie temporal `NOX` en sus componentes: tendencia, estacionalidad y residuo.
En el parámetro ``period`` vamos a poner 365 ya que queremos ver la estacionalidad anual.

In [ ]:
columna_interes = 'NOX'
serie_temporal = df_concatenado_diario[columna_interes]

nox_dec = sm.tsa.seasonal_decompose(serie_temporal, model="additive", period=365)

fig = nox_dec.plot()
fig.set_size_inches((16, 12))

plt.tight_layout()
plt.show()

Muestra que la serie tiene una clara tendencia a la baja, una estacionalidad notable con picos y valles regulares, y los residuos no parecen tener una estructura significativa

### Visualización de la Autocorrelación (ACF)

Vamos a visualizar la autocorrelación de la variable `NOX`, para ello primero vamos a probar con lags bajos y luego con lags más altos.

In [ ]:
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sm.graphics.tsa.plot_acf(df_concatenado_diario['NOX'], lags=1, ax=plt.gca())
plt.title('Función de Autocorrelación (ACF) - Lags=1')
plt.xlabel('Lags')
plt.ylabel('Autocorrelación')

plt.subplot(1, 2, 2)
sm.graphics.tsa.plot_acf(df_concatenado_diario['NOX'], lags=2, ax=plt.gca())
plt.title('Función de Autocorrelación (ACF) - Lags=2')
plt.xlabel('Lags')
plt.ylabel('Autocorrelación')

plt.tight_layout()
plt.show()


Como podemos apreciar con lags bajos tenemos valores altos, lo que indica que los valores recientes están altamente correlacionados con los próximos valores. Esto sugiere que la serie tiene una componente autoregresiva.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
acf_values = sm.tsa.stattools.acf(df_concatenado_diario['NOX'], nlags=50)

lags = np.arange(len(acf_values))

for lag, value in zip(lags, acf_values):
    if lag % 7 == 0 and lag != 0:
        ax.vlines(lag, 0, value, color='red', lw=2)
        ax.plot(lag, value, 'o', color='red')  
    else:
        ax.vlines(lag, 0, value, color='blue', lw=2)  
        ax.plot(lag, value, 'o', color='blue')  

ax.axhline(0, color='black', lw=1)  

plt.title('Función de Autocorrelación (ACF)')
plt.xlabel('Lags')
plt.ylabel('Autocorrelación')
plt.show()

Como se puede apreciar en la gráfica anterior, existe una componente estacional con correlación positiva cada 7/8 lags, que tiende a disminuir con el tiempo.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
acf_values = sm.tsa.stattools.acf(df_concatenado_diario['NOX'], nlags=2000)

lags = np.arange(len(acf_values))  

for lag, value in zip(lags, acf_values):
    if lag % 365 == 0 and lag != 0:  
        ax.vlines(lag, 0, value, color='red', lw=2)  
    ax.plot(lag, value, 'o', color='blue')  

ax.axhline(0, color='black', lw=1) 


plt.title('Función de Autocorrelación (ACF)')
plt.xlabel('Lags')
plt.ylabel('Autocorrelación')
plt.show()

Como podemos ver cada 365 lags, es decir, cada año, existe un pico en el que la correlación es más positiva mostrando cierta estacionalidad anual, aunque la tendencia es a disminuir.

## Partición Entrenamiento y Test

En la partición de los datos, vamos a utilizar una partición `Holdout` con un 90% de los datos para entrenamiento y un 10% para test.

In [ ]:
#guardame el csv de df_concatenado_diario

df_concatenado_diario.to_csv(f"{rutaBase}{nombreComun}2017_2022_diario.csv", index=True)

In [ ]:
porcentaje_entrenamiento = 0.9
df_train = df_concatenado_diario[:int(len(df_concatenado_diario) * porcentaje_entrenamiento)]
df_test = df_concatenado_diario[int(len(df_concatenado_diario) * porcentaje_entrenamiento):]

print(f'Con el porcentaje de %.2f tenemos:' %porcentaje_entrenamiento)
print(f'Tamaño del conjunto de training es %i' %len(df_train))
print(f'Tamaño del conjunto de test es %i' %len(df_test))
print(f'El conjunto de training va de {min(df_train.index)} y {max(df_train.index)}')
print(f'El conjunto de test va de {min(df_test.index)} y {max(df_test.index)}')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
df_train['NOX'].plot(ax=ax, label='train')
df_test['NOX'].plot(ax=ax, label='test')
ax.legend(labels=['Train', 'Test'], loc='best')
ax.set_xlabel('Año')
ax.set_ylabel('NOX')
ax.set_title('NOX mensual')
fig.tight_layout()
plt.show()

Separamos las variables independientes de las dependientes en ambos conjuntos.

En la práctica, se pide en primer lugar realizar el análisis y la obtención de modelos para predecir la variable `NOX` utilizando únicamente los datos de calidad del aire. Luego, se pide también volver a hacer el análisis, pero ampliando el dataset con los datos meteorológicos. Por esta razón, hemos creado dos datasets distintos, uno que contiene solo los datos sobre calidad del aire (`df_concatenado`), y otro al que le añadimos además los datos meteorológicos (`df_concatenado_aire`):

In [ ]:
X_train = df_train.drop(columns = 'NOX')
y_train = df_train['NOX']

X_test = df_test.drop(columns='NOX')
y_test = df_test['NOX']

In [ ]:
columnas_variables_metereologicas = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS']

X_train_aire = X_train.drop(columns=columnas_variables_metereologicas)
y_train_aire = df_train['NOX']

X_test_aire = X_test.drop(columns=columnas_variables_metereologicas)
y_test_aire = df_test['NOX']

In [ ]:
train_NOX = df_train['NOX']
test_NOX = df_test['NOX']

## Transformaciones

En ocasiones, aplicar ciertas transformaciones a los valores de una serie puede ayudar a generar mejores modelos de predicción. Algunas de estas transformaciones posibles son:

- Raíz cuadrada: La transformación mediante la raíz cuadrada se puede utilizar para estabilizar la varianza. Se calcula como:

$$\hat X_t = \sqrt{X_t}$$

- Logarítmica: La transformación logarítmica tiene el mismo efecto que la raíz cuadrada, aunque en algunos casos puede ser más interpretable. Se calcula como:

$$\hat X_t = \ln{X_t}$$

- Box-Cox: La transformación Box-Cox puede ser útil a la hora de generar modelos de predicción, ya que tiene como objetivo aproximar los valores de las variables a una distribución normal. Esta transformación se define como:

$$
\hat X_t(\lambda) =
\begin{cases}
\frac{y^\lambda - 1}{\lambda} & \text{si }\lambda \neq 0 \\
\ln(y) & \text{si }\lambda = 0
\end{cases}
$$

A continuación, mostramos algunos estadísticos para las variables del dataset. Estos estadísticos incluyen:

- `count`: número total de datos en cada columna.
- `mean`: media de los valores de cada variable.
- `std`: desviación estándar, que mide la dispersión de los datos respecto a la media.
- `min`: valor mínimo de cada columna.
- `25%`: percentil 25 o primer cuartil, que es el valor por debajo del cual se encuentra el 25% de los datos.
- `50%`: percentil 50 o mediana, que es el valor por debajo del cual se encuentra el 50% de los datos.
- `75%`: percentil 75 o tercer cuartil, que es el valor por debajo del cual se encuentra el 75% de los datos.
- `max`: valor máximo de cada columna.

In [ ]:
df_concatenado_diario.describe()

Al analizar la estadística descriptiva de los datos, se pueden extraer algunos argumentos que justifican la necesidad de aplicar alguna transformación:

- Algunas variables, como `PM10`, tienen una gran diferencia entre los valores mínimos y los máximos (rango de $5.29$ a $271.92$) lo cual, observando además los valores de los percentiles (por ejemplo, el percentil 75 es de $31.12$, bastante inferior al máximo), sugiere que la distribución de esta variable podría estar sesgada positivamente. Usando por ejemplo la transformación logarítmica o la Box-Cox, podremos conseguir que estas distribuciones se asemejen más a una distribución normal.

- Algunas variables como `DD` o `RS` tienen una desviación estándar muy alta ($103.01$ y $2050.74$ respectivamente), y por tanto, la elevada variabilidad de estos valores podría dificultar la modelización. Por ejemplo, la transformación de la raíz cuadrada, la logarítmica o la Box-Cox podría conseguir datos más uniformes y manejables.

- Los percentiles muestran grandes saltos entre los cuartiles. Por ejemplo, en `PM10`, el rango intercuartil (IQR) es $31.12−18=13.12$, pero la diferencia entre el mínimo y el máximo es mucho mayor ($271.92−5.29=266.63$). Esto sugiere que hay valores extremos que podrían influir excesivamente en los modelos. Usando por ejemplo la transformación de la raíz cuadrada, la logarítmica o la Box-Cox, podemos mitigar este efecto.

Tras observar estos resultados, vamos a realizar un análisis más detallado, variable por variable, para estudiar qué transformación puede ser más conveniente aplicar para cada variable.

In [ ]:
df_concatenado_diario_transformed = df_concatenado_diario.copy()

### NO

Para determinar qué transformación podría ser más adecuada para la variable `NO`, comenzaremos analizando sus estadísticas descriptivas y características.

En primer lugar, se puede observar que la variable `NO` presenta una cierta **asimetría**, dado que, como su coeficiente de simetría es $B_1 = 2.17 > 0$, entonces su distribución es sesgada positiva, lo que significa que la cola superior es más larga y por tanto, hay mayor concentración de datos al principio de la distribución. El coeficiente de simetría se calcula con la siguiente fórmula (aunque en este caso lo calcularemos con el método `.skew()` de `pandas`):

$$ B_1 = \frac{\sum_{i=1}^n (x_i - \bar X)^3}{(n-1) S^3}$$

donde $\bar X$ es la media muestral, $S$ es la desviación estándar muestral y $n$ es el número de observaciones.

In [ ]:
# Calcular el coeficiente de simetría de la variable 'NO'
coeficiente_simetria = df_concatenado_diario['NO'].skew()
print(f"Coeficiente de simetría para la variable 'NO': {coeficiente_simetria}")

Esto mismo lo podemos apreciar en el siguiente gráfico, que muestra la distribución de la variable `NO`, donde efectivamente se observa una cola superior más larga:

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['NO'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'NO'", fontsize=16)
plt.xlabel("Valores de 'NO'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Además, volviendo a los estadísticos estudiados anteriormente, vemos que el valor máximo $17.375$ de la variable `NO` es bastante superior a su media $3.647677$, lo cual podría indicar la presencia de **outliers**.

Viendo estos resultados, la transformación logarítmica podría ayudar a abordar el problema de la asimetría. Sin embargo, dado que la transformación logarítmica es tan solo un caso particular de la transformación Box-Cox, tomando $\lambda = 0$, vamos a probar a aplicar la transformación Box-Cox. De esta forma, podremos encontrar el parámetro $\lambda$ que da la mejor transformación, en lugar de fijarlo directamente como $\lambda = 0$.

In [ ]:
# Aplica la transformación Box-Cox
NO_boxcox, lambda_NO = boxcox(df_concatenado_diario['NO'])

A continuación, para analizar si la transformación ha sido efectiva, vamos a graficar los histogramas con la distribución de la variable `NO` antes y después de la transformación, y vamos a compararlos.

In [ ]:
# Crear el gráfico de distribución para la variable 'NO'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'NO'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['NO'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'NO' (Original)", fontsize=16)
plt.xlabel("Valores de 'NO'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'NO_boxcox'
plt.subplot(1, 2, 2)
sns.histplot(NO_boxcox, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'NO' (Transformación Box-Cox)", fontsize=16)
plt.xlabel("Valores de 'NO' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Vemos que tras aplicar la transformación Box-Cox a la variable `NO`, la distribución de la variable se vueve más **simétrica**, y se reduce la cola hacia la derecha. Se aprecia también que la distribución de la variable transformada se acerca más a una distribución **normal** que la de la variable original.

Por otro lado, también observamos que se reduce el rango de los datos y la **varianza** parece más **estabilizada**, sin haber valores altos tan dispersos como en la distribución de la variable original.

Para asegurarnos de que efectivamente, la distribución se asemeja más a la normal tras la transformación, podemos mostrar el Q-Q plot de la variable `NO` original y de la transformada, y compararlos.

In [ ]:
# Crear el Q-Q plot para la variable 'NO'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'NO'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['NO'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'NO' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'NO_boxcox'
plt.subplot(1, 2, 2)
stats.probplot(NO_boxcox, dist="norm", plot=plt)
plt.title("Q-Q plot de 'NO' (Transformación Box-Cox)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Observando estos gráficos Q-Q plot, efectivamente se confirma que la variable `NO` tarnsformada mediante transformación Box-Cox sigue una distribución que se aproxima más a la distribución normal que la variable original sin transformar.

Con todo esto, podemos concluir que la transformación ha sido efectiva, y ha mejorado la distribución de la variable.

In [ ]:
df_concatenado_diario_transformed['NO'] = NO_boxcox

### NO2

Para buscar la mejor transformación para la variable `NO2`, en primer lugar observamos que hay una ligera diferencia entre la media ($11.261764$) y la mediana ($9.875$), lo cual parece indicar una ligera **asimetría** en la distribución de la variable. Para confirmar esto, podemos de nuevo calcular el coeficiente de simetría, que en este caso nos da $B_1 = 1.31 > 0$, por lo que nuevamente la distribución es sesgada positiva.

In [ ]:
# Calcular el coeficiente de simetría de la variable 'NO2'
coeficiente_simetria = df_concatenado_diario['NO2'].skew()
print(f"Coeficiente de simetría para la variable 'NO2': {coeficiente_simetria}")

Podemos además mostrar un histograma con la distribución de la variable, donde se observa que hay una cola superior más larga. Aunque en este caso dicha asimetría no es tan extrema como con la variable `NO` anterior, igualmente podría ser beneficioso intentar aproximar la distribución de la variable `NO2` por una distribución normal.

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['NO2'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'NO2'", fontsize=16)
plt.xlabel("Valores de 'NO2'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Además, la desviación estándar $5.356303$ es relativamente alta, lo que sugiere que los datos pueden ser dispersos. El valor máximo $40.958333	$ está bastante alejado de la media $11.261764$, lo cual es consistente con la presencia de una cola a la derecha.

Como en este caso, el sesgo positivo observado no es tan pronunciado como en `NO`, la transformación logarítmica podría ser suficiente para abordar el problema de la cola larga a la derecha y estabilizar la varianza, logrando una distribución aproximada a la normal sin necesidad de ajustes complejos como Box-Cox. Por ello, con la variable `NO2` vamos aprobar a aplicar la transformación logarítmica y ver si da buenos resultados.

In [ ]:
# Aplica la transformación logarítmica
NO2_log = np.log(df_concatenado_diario['NO2'])

Graficamos los histogramas de la variable `NO2` con y sin transformación.

In [ ]:
# Crear el gráfico de distribución para la variable 'NO2'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'NO2'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['NO2'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'NO2' (Original)", fontsize=16)
plt.xlabel("Valores de 'NO2'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'NO2_log'
plt.subplot(1, 2, 2)
sns.histplot(NO2_log, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'NO2' (Transformación logarítmica)", fontsize=16)
plt.xlabel("Valores de 'NO2' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Vemos que tras aplicar la transformación logarítmica a la variable `NO2`, su distribución se vueve más **simétrica**, además de reducirse la cola hacia la derecha. También se reduce considerablemente el rango de la variable, y su **varianza** se **estabiliza**, sin haber valores altos tan dispersos como en la distribución de la variable original.

Por otro lado, la distribución de la variable transformada parece acercarse más a una distribución **normal** que la de la variable original. Para asegurarnos de esto, mostramos el Q-Q plot de la variable `NO2` original y de la transformada, y los comparamos.

In [ ]:
# Crear el Q-Q plot para la variable 'NO2'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'NO2'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['NO2'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'NO2' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'NO2_log'
plt.subplot(1, 2, 2)
stats.probplot(NO2_log, dist="norm", plot=plt)
plt.title("Q-Q plot de 'NO2' (Transformación logarítmica)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Al observar los Q-Q plot, podemos apreciar que gracias a la transformación logarítmica, la distribución de `NO2` se aproxima considerablemente por una distribución normal.

Tras este análisis, podemos concluir que la transformación logarítmica ha sido efectiva, y no es necesario aplicar otras transformaciones más complejas.

In [ ]:
df_concatenado_diario_transformed['NO2'] = NO2_log

### SO2 

Para esta variable, observamos en primer lugar que la media ($6.834639$) y la mediana ($6.375$) toman valores muy similares, lo cual podría sugerir que `SO2` tiene una distribución algo más simétrica que las dos variables estudiadas con anterioridad. De hecho, al calcular el coeficiente de simetría se obtiene $B_1 = 0.69 > 0$, lo cual indica una ligera asímetría, aunque al ser un valor cercano a $0$ no es tan pronunciada como en los casos anteriores.

In [ ]:
# Calcular el coeficiente de simetría de la variable 'SO2'
coeficiente_simetria = df_concatenado_diario['SO2'].skew()
print(f"Coeficiente de simetría para la variable 'SO2': {coeficiente_simetria}")

Además, aunque se observa una cola a la derecha, esta no parece ser tan marcada como en los casos anteriores, tal como se puede apreciar en el histograma que se muestra a continuación:

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['SO2'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'SO2'", fontsize=16)
plt.xlabel("Valores de 'SO2'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

La desviación estándar $4.024215$ sugiere una cierta dispersión en los datos. Además, hay ciertos valores altos que podrían ser outliers. Observando la distribución de la variable, no se aprecia claramente qué transformación podría funcionar mejor, por lo que vamos a probar a aplicar las tres transformaciones y las compararemos, estudiando cuál de ellas ofrece mejores resultados. Comenzamos con la transformación de raíz cuadrada:

In [ ]:
# Aplica la transformación de raíz cuadrada
SO2_sqrt = np.sqrt(df_concatenado_diario['SO2'])

Mostramos a continuación los histogramas con las distribuciones de la variable `SO2` antes y después de la transformación de raíz cuadrada.

In [ ]:
# Crear el gráfico de distribución para la variable 'SO2'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'SO2'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['SO2'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'SO2' (Original)", fontsize=16)
plt.xlabel("Valores de 'SO2'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'SO2_sqrt'
plt.subplot(1, 2, 2)
sns.histplot(SO2_sqrt, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'SO2' (Transformación de raíz cuadrada)", fontsize=16)
plt.xlabel("Valores de 'SO2' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Observando la distribución de la variable transformada, no parece que se aproxime por una normal. Sin embargo, sí se consigue reducir el rango de la variable, así como la asimetría de su distribución. Representemos los Q-Q plot para comprobar si se mejora la normalidad al aplicar la transformación:

In [ ]:
# Crear el Q-Q plot para la variable 'SO2'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'SO2'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['SO2'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'SO2' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'SO2_sqrt'
plt.subplot(1, 2, 2)
stats.probplot(SO2_sqrt, dist="norm", plot=plt)
plt.title("Q-Q plot de 'SO2' (Transformación de raíz cuadrada)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Observando los Q-Q plot, se observa una ligera mejora en cuanto a la normalidad respecto de la variable original, aunque no parece que la variable transformada con raíz cuadrada siga tampoco una distribución normal. Probemos a aplicar el test de Shapiro-Wilk para hacer una comprobación más precisa de la normalidad:

In [ ]:
stat, p = shapiro(SO2_sqrt)
print(f'Estadístico: {stat}, p-valor: {p}')

if p > 0.05:
    print("La distribución es aproximadamente normal.")
else:
    print("La distribución no es normal.")

Por tanto, mediante la transformación de la raíz cuadrada no se logra una distribución normal para la variable `SO2`.

Dado que los resultados en cuanto a normalidad no parecen ser satisfactorios, vamos a probar a analizar otras métricas como la asimetría y la curtosis.

In [ ]:
print(f"Asimetría raíz cuadrada: {skew(SO2_sqrt)}")
print(f"Curtosis raíz cuadrada: {kurtosis(SO2_sqrt)}")

La asimetría obtenida $0.23$ es ligeramente positiva, pero al ser tan cercana a $0$, sugiere que la distribución es casi simétrica. Por otro lado, la curtosis es negativa, lo que indica que la distribución es más plana que la distribución normal.

Probemos ahora con la transformación logarítmica:

In [ ]:
# Aplica la transformación logarítmica
SO2_log = np.log(df_concatenado_diario['SO2'])

Mostramos los histogramas con las distribuciones de la variable original y de la variable transformada:

In [ ]:
# Crear el gráfico de distribución para la variable 'SO2'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'SO2'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['SO2'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'SO2' (Original)", fontsize=16)
plt.xlabel("Valores de 'SO2'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'SO2_log'
plt.subplot(1, 2, 2)
sns.histplot(SO2_log, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'SO2' (Transformación logarítmica)", fontsize=16)
plt.xlabel("Valores de 'SO2' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

De nuevo, mediante la transformación logarítmica tampoco parece lograrse una aproximación a la distribución normal, aunque sí se consigue reducir el rango y la asimetría.

A continuación mostramos los gráficos Q-Q plot:

In [ ]:
# Crear el Q-Q plot para la variable 'SO2'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'SO2'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['SO2'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'SO2' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'SO2_log'
plt.subplot(1, 2, 2)
stats.probplot(SO2_log, dist="norm", plot=plt)
plt.title("Q-Q plot de 'SO2' (Transformación logarítmica)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

En este caso se observa que la distribución parece alejarse todavía más de la distribución normal, en lugar de aproximarse a la misma. Esto mismo se comprueba con el test de Shapiro-Wilk:

In [ ]:
stat, p = shapiro(SO2_log)
print(f'Estadístico: {stat}, p-valor: {p}')

if p > 0.05:
    print("La distribución es aproximadamente normal.")
else:
    print("La distribución no es normal.")

In [ ]:
print(f"Asimetría log: {skew(SO2_log)}")
print(f"Curtosis log: {kurtosis(SO2_log)}")

Con la transformación logarítmica, se obtiene una asímetría negativa, y mayor que la obtenida con la transformación de raíz cuadrada. Además, la curtosis vuelve a ser negativa, pero más plana que con la transformación de raíz cuadrada.

Por lo tanto, vamos a descartar la transformación logarítmica, ya que es la que peores resultados ha dado hasta el momento en cuanto a normalidad, asimetría y curtosis.

Por último, probamos a aplicar la transformación Box-Cox y analizamos si se consigue alguna mejora considerable.

In [ ]:
# Aplica la transformación Box-Cox
SO2_boxcox, lambda_SO2 = boxcox(df_concatenado_diario['SO2'])

Mostramos los histogramas de las distribuciones de la variable original y la transformada:

In [ ]:
# Crear el gráfico de distribución para la variable 'SO2'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'SO2'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['SO2'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'SO2' (Original)", fontsize=16)
plt.xlabel("Valores de 'SO2'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'SO2_boxcox'
plt.subplot(1, 2, 2)
sns.histplot(SO2_boxcox, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'SO2' (Transformación Box-Cox)", fontsize=16)
plt.xlabel("Valores de 'SO2' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

De nuevo, la distribución de la variable transformada no parece aproximarse a una distribución normal, aunque sí se consigue reducir el rango y la asimetría.

Mostramos a continuación los gráficos Q-Q plot:

In [ ]:
# Crear el Q-Q plot para la variable 'SO2'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'SO2'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['SO2'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'SO2' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'SO2_boxcox'
plt.subplot(1, 2, 2)
stats.probplot(SO2_boxcox, dist="norm", plot=plt)
plt.title("Q-Q plot de 'SO2' (Transformación Box-Cox)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Parece haber una ligera mejora tras aplicar la transformación, aunque no parece demasiado significativa, ni se logra una distribución cercana a la normal, como se puede comprobar con el test de Shapiro-Wilk:

In [ ]:
stat, p = shapiro(SO2_boxcox)
print(f'Estadístico: {stat}, p-valor: {p}')

if p > 0.05:
    print("La distribución es aproximadamente normal.")
else:
    print("La distribución no es normal.")

In [ ]:
print(f"Asimetría Box-Cox: {skew(SO2_boxcox)}")
print(f"Curtosis Box-Cox: {kurtosis(SO2_boxcox)}")

En este caso, la asimetría es ligeramente negativa, pero es casi simétrica, y de todas las transformacioens probadas, la Box-Cox es la que ofrece un mejor ajuste. Por otro lado, en cuanto a la curtosis, toma un valor negativo, y ligeramente peor que con la transformación de raíz cuadrada, aunque sin haber mucha diferencia entre ambas.

Como conclusión final, se tiene que:

- La que mejor aproxima a la distribución normal es la transformación de raíz cuadrada, aunque con una diferencia mínima con la Box-Cox.
- La variable transformada con raíz cuadrada consigue un rango menor que la transformada con Box-Cox.
- La transformación Box-Cox consigue una mejor simetría, pero con poca diferencia con la raíz cuadrada.
- La transformación de raíz cuadrada consigue un valor ligeramente mejor para la curtosis.

Con base en estas observaciones, nos vamos a quedar con la transformación de raíz cuadrada, dado que las diferencias con Box-Cox son mínimas, pero ofrece ventajas clave como una mejor curtosis y un rango más estrecho.

In [ ]:
df_concatenado_diario_transformed['SO2'] = SO2_sqrt

### PM10

Analizando los estadísticos de la variable `PM10`, observamos que esta tiene un rango muy amplio, ya que el mínimo $5.291667$ y el máximo $271.916667$ se encuentran muy separados, lo cual puede dificultar la modelización. Además, la desviación estándar $13.965115$ es bastante elevada, lo que muestra una dispersión considerable.

Por otro lado, como el máximo es considerablemente mayor que la mediana ($23.458333$), la distribución parece estar sesgada hacia la derecha. Esto se puede confirmar al calcular el coeficiente de simetría $B_1 = 4.77 > 0$, el cual es bastante elevado, indicando una fuerte asimetría.

In [ ]:
# Calcular el coeficiente de simetría de la variable 'PM10'
coeficiente_simetria = df_concatenado_diario['PM10'].skew()
print(f"Coeficiente de simetría para la variable 'PM10': {coeficiente_simetria}")

Efectivamente, al mostrar el histograma con la distribución de la variable, se aprecia una cola larga a la derecha.

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['PM10'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'PM10'", fontsize=16)
plt.xlabel("Valores de 'PM10'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Al tener una distribución altamente sesgada, pasamos directamente a probar con la transformación Box-Cox, que puede ajustar el parámetro $\lambda$ para ser más efectiva ante este nivel de sesgo.

In [ ]:
# Aplica la transformación Box-Cox
PM10_boxcox, lambda_PM10 = boxcox(df_concatenado_diario['PM10'])

Mostramos los histogramas con las distribuciones de la variable `PM10` con y sin la transformación Box-Cox:

In [ ]:
# Crear el gráfico de distribución para la variable 'PM10'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'PM10'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['PM10'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'PM10' (Original)", fontsize=16)
plt.xlabel("Valores de 'PM10'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'PM10_boxcox'
plt.subplot(1, 2, 2)
sns.histplot(PM10_boxcox, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'PM10' (Transformación Box-Cox)", fontsize=16)
plt.xlabel("Valores de 'PM10' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Observamos que los resultados son bastante buenos, pues la distribución parece aproximarse bastante bien por una normal, además de haberse reducido considerablemente el rango de la variable, y haberse vuelto mucho más simétrica.

Podemos mostrar los gráficos Q-Q plot para asegurarnos de que la distribución de la variable transformada se aproxima considerablemente a la distribución normal:

In [ ]:
# Crear el Q-Q plot para la variable 'PM10'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'PM10'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['PM10'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'PM10' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'PM10_boxcox'
plt.subplot(1, 2, 2)
stats.probplot(PM10_boxcox, dist="norm", plot=plt)
plt.title("Q-Q plot de 'PM10' (Transformación Box-Cox)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Efectivamente, observando estos gráficos, se observa una gran mejora tras la transformación Box-Cox en cuanto a la normalidad de la distribución de la variable `PM10`.

In [ ]:
df_concatenado_diario_transformed['PM10'] = PM10_boxcox

### TMP

Analizando los estadísticos de la variable `TMP` se puede apreciar que la desviación estándar $5.837991$ es ligeramente alta, aunque no tan exagerada como en otras variables estudiadas con anterioridad. Por otro lado, el rango es razonablemente amplio, con un mínimo de $7.445652$ y un máximo de $33.916667$, pero no excesivo.

Además, calculando el coeficiente de simetría se obtiene $B_1 = 0.07$, lo cual es bastante bajo, indicando que la distribución no está excesivamente sesgada.

In [ ]:
# Calcular el coeficiente de simetría de la variable 'TMP'
coeficiente_simetria = df_concatenado_diario['TMP'].skew()
print(f"Coeficiente de simetría para la variable 'TMP': {coeficiente_simetria}")

Analizando el histograma, se pueden observar dos máximos en el mismo, lo cual sugiere que podrían existir dos grupos diferentes dentro de los datos (por ejemplo, temperaturas de diferentes épocas del año).

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['TMP'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'TMP'", fontsize=16)
plt.xlabel("Valores de 'TMP'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Comprobemos mediante el test de Shapiro-Wilk si la distribución es normal:

In [ ]:
stat, p = shapiro(df_concatenado_diario['TMP'])
print(f'Estadístico: {stat}, p-valor: {p}')

if p > 0.05:
    print("La distribución es aproximadamente normal.")
else:
    print("La distribución no es normal.")

Dado que la transformación Box-Cox es más flexible, ya que ajusta su parámetro $\lambda$ para manejar mejor los sesgos y la dispersión, vamos a probar a aplicar esta transformación para ver si de esta forma conseguimos resultados efectivos, incluso habiendo dos máximos en la distribución.

In [ ]:
# Aplica la transformación Box-Cox
TMP_boxcox, lambda_TMP = boxcox(df_concatenado_diario['TMP'])

Mostramos los histogramas para la variable original y la variable transformada:

In [ ]:
# Crear el gráfico de distribución para la variable 'TMP'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'TMP'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['TMP'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'TMP' (Original)", fontsize=16)
plt.xlabel("Valores de 'TMP'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'TMP_boxcox'
plt.subplot(1, 2, 2)
sns.histplot(TMP_boxcox, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'TMP' (Transformación Box-Cox)", fontsize=16)
plt.xlabel("Valores de 'TMP' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

A pesar de que sigue habiendo dos máximos, al menos se logra reducir el rango de la variable. Sin embargo, no se logra una distribución normal, aunque mirando los gráficos Q-Q plot, parece apreciarse una muy ligera mejora en este sentido:

In [ ]:
# Crear el Q-Q plot para la variable 'TMP'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'TMP'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['TMP'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'TMP' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'PM10_boxcox'
plt.subplot(1, 2, 2)
stats.probplot(TMP_boxcox, dist="norm", plot=plt)
plt.title("Q-Q plot de 'TMP' (Transformación Box-Cox)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Esto mismo puede comprobarse con el test de Shapiro-Wilk:

In [ ]:
stat, p = shapiro(TMP_boxcox)
print(f'Estadístico: {stat}, p-valor: {p}')

if p > 0.05:
    print("La distribución es aproximadamente normal.")
else:
    print("La distribución no es normal.")

Como conclusión, aunque la transformación no consigue aproximar por una distribución normal, sí se logra una ligera mejora en cuanto al rango y la varianza de la variable, por lo que puede ser conveniente aplicar la transformación Box-Cox a la variable `TMP`.

In [ ]:
df_concatenado_diario_transformed['TMP'] = TMP_boxcox

### HR

La desviación estándar es $14.474146$, lo que sugiere que hay una variabilidad significativa en los valores de `HR` alrededor de su media. Esto es relativamente alto, indicando que los valores de humedad relativa pueden fluctuar considerablemente.

El valor mínimo de `HR` es $32.125000$, lo que muestra que, en el conjunto de datos, no hay valores de `HR` demasiado bajos, lo cual es importante porque no hay anomalías extremas en este sentido.

La media ($71.218916$) está cerca de la mediana ($72.541667$), lo que podría indicar que la distribución no está extremadamente sesgada. Esto mismo se puede comprobar con el coeficiente de simetría $B_1 = -0.26 <0$, que al ser negativo podría indicar una ligera simetría negativa, es decir, sesgo hacia la izquierda, aunque la asimetría no es muy pronunciada, ya que este valor está bastante cerca de $0$.

In [ ]:
# Calcular el coeficiente de simetría de la variable 'HR'
coeficiente_simetria = df_concatenado_diario['HR'].skew()
print(f"Coeficiente de simetría para la variable 'HR': {coeficiente_simetria}")

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['HR'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'HR'", fontsize=16)
plt.xlabel("Valores de 'HR'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
stat, p = shapiro(df_concatenado_diario['HR'])
print(f'Estadístico: {stat}, p-valor: {p}')

if p > 0.05:
    print("La distribución es aproximadamente normal.")
else:
    print("La distribución no es normal.")

Vamos a probar con la distribución Box-Cox, para ver si conseguimos aproximar por una distribución normal.

In [ ]:
# Aplica la transformación Box-Cox
HR_boxcox, lambda_HR = boxcox(df_concatenado_diario['HR'])

In [ ]:
# Crear el gráfico de distribución para la variable 'HR'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'HR'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['HR'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'HR' (Original)", fontsize=16)
plt.xlabel("Valores de 'HR'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'HR_boxcox'
plt.subplot(1, 2, 2)
sns.histplot(HR_boxcox, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'HR' (Transformación Box-Cox)", fontsize=16)
plt.xlabel("Valores de 'HR' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Si bien se ha conseguido reducir la asimetría de la distribución, también podemos apreciar que el rango de valores que obtenemos, entre $100$ y $500$, es claramente excesivo y no parece apropiado para la interpretación o el modelado.

Por ello, vamos a probar a aplicar una transformación logarítmica:

In [ ]:
# Aplica la transformación logarítmica
HR_log = np.log(df_concatenado_diario['HR'])

In [ ]:
# Crear el gráfico de distribución para la variable 'HR'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'HR'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['HR'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'HR' (Original)", fontsize=16)
plt.xlabel("Valores de 'HR'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'HR_log'
plt.subplot(1, 2, 2)
sns.histplot(HR_log, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'HR' (Transformación logarítmica)", fontsize=16)
plt.xlabel("Valores de 'HR' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

En este caso, vemos que al aplicar la transformación logarítmica, aumenta la asimetría de la variable, por lo que no parece una transformación adecuada.

Por último, vamos a probar con la transformación de raíz cuadrada:

In [ ]:
# Aplica la transformación de raíz cuadrada
HR_sqrt = np.sqrt(df_concatenado_diario['HR'])

In [ ]:
# Crear el gráfico de distribución para la variable 'HR'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'HR'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['HR'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'HR' (Original)", fontsize=16)
plt.xlabel("Valores de 'HR'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'HR_sqrt'
plt.subplot(1, 2, 2)
sns.histplot(HR_sqrt, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'HR' (Transformación de raíz cuadrada)", fontsize=16)
plt.xlabel("Valores de 'HR' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Con la transformación de raíz cuadrada, vemos que se consigue reducir el rango de la variable a un rango más razonable, sin afectar demasiado a la asimetría, y aunque no parece lograrse una distribución totalmente normal, tampoco parece estar tan alejada de la misma, como podemos comprobar con el Q-Q plot:

In [ ]:
# Crear el Q-Q plot para la variable 'HR'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'HR'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['HR'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'HR' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'HR_sqrt'
plt.subplot(1, 2, 2)
stats.probplot(HR_sqrt, dist="norm", plot=plt)
plt.title("Q-Q plot de 'HR' (Transformación de raíz cuadrada)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Como conclusión, nos vamos a quedar finalmente con la transformación de raíz cuadrada.

In [ ]:
df_concatenado_diario_transformed['HR'] = HR_sqrt

### PRB

Como podemos observar en la gráfica observamos que la variable `PRB` sigue una distribución normal, por lo que no creemos necesario aplicar ninguna transformación.

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['PRB'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'PRB'", fontsize=16)
plt.xlabel("Valores de 'PRB'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### VV

El valor medio de `VV` es $1.280236$, lo que indica que en promedio los valores de esta variable son bastante cercanos a $1$, pero pueden variar ligeramente, debido a que la desviación estándar es $0.269307$, lo que sugiere que la mayoría de los valores están relativamente cercanos a la media, pero también existe algo de variabilidad.

In [ ]:
# Calcular el coeficiente de simetría de la variable 'VV'
coeficiente_simetria = df_concatenado_diario['VV'].skew()
print(f"Coeficiente de simetría para la variable 'VV': {coeficiente_simetria}")

Dado que el coeficiente de simetría para la variable VV es $B_1 = 1.74 > 0$, podemos concluir que la distribución de esta variable tiene una asimetría positiva considerable, lo que significa que tiene una cola derecha más alargada. En este caso, es importante considerar una transformación que ayude a reducir esa asimetría y acercar la distribución a una forma más simétrica, o incluso normal, si es posible.

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['VV'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'VV'", fontsize=16)
plt.xlabel("Valores de 'VV'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Probamos en primer lugar con una trasnformación de raíz cuadrada:

In [ ]:
# Aplica la transformación de raíz cuadrada
VV_sqrt = np.sqrt(df_concatenado_diario['VV'])

In [ ]:
# Crear el gráfico de distribución para la variable 'VV'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'VV'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['VV'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'VV' (Original)", fontsize=16)
plt.xlabel("Valores de 'VV'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'VV_sqrt'
plt.subplot(1, 2, 2)
sns.histplot(VV_sqrt, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'VV' (Transformación de raíz cuadrada)", fontsize=16)
plt.xlabel("Valores de 'VV' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Tras la transformación, la variable sigue siendo bastante asimétrica, y no muestra mejoras considerables.

In [ ]:
# Crear el Q-Q plot para la variable 'VV'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'VV'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['VV'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'VV' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'VV_sqrt'
plt.subplot(1, 2, 2)
stats.probplot(VV_sqrt, dist="norm", plot=plt)
plt.title("Q-Q plot de 'VV' (Transformación de raíz cuadrada)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Probamos ahora con una transformación logarítmica:

In [ ]:
# Aplica la transformación logarítmica
VV_log = np.log(df_concatenado_diario['VV'])

In [ ]:
# Crear el gráfico de distribución para la variable 'VV'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'VV'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['VV'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'VV' (Original)", fontsize=16)
plt.xlabel("Valores de 'VV'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'VV_log'
plt.subplot(1, 2, 2)
sns.histplot(VV_log, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'VV' (Transformación logarítmica)", fontsize=16)
plt.xlabel("Valores de 'VV' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Esta transformación sigue sin resolver el problema de la asimetría, aunque observando el Q-Q plot parece que mejora ligeramente en cuanto a la normalidad: 

In [ ]:
# Crear el Q-Q plot para la variable 'VV'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'VV'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['VV'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'VV' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'VV_log'
plt.subplot(1, 2, 2)
stats.probplot(VV_log, dist="norm", plot=plt)
plt.title("Q-Q plot de 'VV' (Transformación logarítmica)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Por último, probamos con la transformación Box-Cox:

In [ ]:
# Aplica la transformación de Box-Cox
VV_boxcox, lambda_VV = boxcox(df_concatenado_diario['VV'])

In [ ]:
# Crear el gráfico de distribución para la variable 'VV'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'VV'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['VV'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'VV' (Original)", fontsize=16)
plt.xlabel("Valores de 'VV'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'VV_boxcox'
plt.subplot(1, 2, 2)
sns.histplot(VV_boxcox, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'V' (Transformación Box-Cox)", fontsize=16)
plt.xlabel("Valores de 'VV' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

La transformación Box-Cox es la que peores resultados ha dado, pues han aparecido huecos por en medio del histograma.

Por tanto, parece que aunque ninguna de las transformaciones tiene un desempeño excelente, la transformación logarítmica es la que ha dado mejores resultados, por lo que nos vamos a quedar con esta.

In [ ]:
df_concatenado_diario_transformed['VV'] = VV_log

### O3

Esta variable tiene un rango bastante grande, desde el mínimo $8.608696$ hasta el máximo $159.00$, por lo que puede ser conveniente reducir dicho rango. 

Por otro lado, no parece mostrar una gran asimetría, pues su coeficiente de simetría $B_1 = 0.199 > 0$ es bastante cercano a $0$.

In [ ]:
# Calcular el coeficiente de simetría de la variable 'O3'
coeficiente_simetria = df_concatenado_diario['O3'].skew()
print(f"Coeficiente de simetría para la variable 'O3': {coeficiente_simetria}")

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['O3'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'O3'", fontsize=16)
plt.xlabel("Valores de 'O3'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Aplicamos una transformación Box-Cox para ver si podemos reducir un poco el rango:

In [ ]:
# Aplica la transformación de Box-Cox
O3_boxcox, lambda_O3 = boxcox(df_concatenado_diario['O3'])

In [ ]:
# Crear el gráfico de distribución para la variable 'O3'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'O3'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['O3'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'O3' (Original)", fontsize=16)
plt.xlabel("Valores de 'O3'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'O3_boxcox'
plt.subplot(1, 2, 2)
sns.histplot(O3_boxcox, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'O3' (Transformación Box-Cox)", fontsize=16)
plt.xlabel("Valores de 'O3' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Vemos que el rango se reduce un poco, y además, aunque la variable original ya se acercaba bastante a una normal, vemos que con la transformación Box-Cox se aprecia una ligera mejora en este sentido también, como se puede observar en el Q-Q plot:

In [ ]:
# Crear el Q-Q plot para la variable 'O3'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'O3'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['O3'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'O3' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'O3_boxcox'
plt.subplot(1, 2, 2)
stats.probplot(O3_boxcox, dist="norm", plot=plt)
plt.title("Q-Q plot de 'O3' (Transformación Box-Cox)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

In [ ]:
df_concatenado_diario_transformed['O3'] = O3_boxcox

### RS

La media es $4272.294776$, lo cual sugiere que, en promedio, los valores de `RS` son altos. Además, el valor de $2050.740902$ para la desviación estándar muestra que hay bastante dispersión en los datos, lo que significa que los valores de `RS` tienen una gran variabilidad.

El valor mínimo es $144.594595$ y el máximo $14407.320611$. Esto sugiere una gran variabilidad en los datos, ya que el rango de valores es bastante extenso.

Además, el coeficiente de simetría $B_1 = 0.395 > 0$ indica que la distribución tiene una ligera asimetría positiva. Es decir, la cola de la distribución es un poco más larga en el lado derecho de la media, aunque no es extremadamente asimétrica.

In [ ]:
# Calcular el coeficiente de simetría de la variable 'RS'
coeficiente_simetria = df_concatenado_diario['RS'].skew()
print(f"Coeficiente de simetría para la variable 'RS': {coeficiente_simetria}")

Esto mismo también puede observarse en el histograma siguiente:

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['RS'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'RS'", fontsize=16)
plt.xlabel("Valores de 'RS'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Probemos en primer lugar con una transformación Box-Cox, ya que es la más flexible de todas, y puede ser eficiente a la hora de aproximar por una distribución normal.

In [ ]:
# Aplica la transformación de Box-Cox
RS_boxcox, lambda_RS = boxcox(df_concatenado_diario['RS'])

In [ ]:
# Crear el gráfico de distribución para la variable 'RS'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'RS'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['RS'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'RS' (Original)", fontsize=16)
plt.xlabel("Valores de 'RS'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'RS_boxcox'
plt.subplot(1, 2, 2)
sns.histplot(RS_boxcox, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'RS' (Transformación Box-Cox)", fontsize=16)
plt.xlabel("Valores de 'RS' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Vemos que el rango de valores, aunque se reduce, sigue siendo considerablemente alto, por lo que tal vez pueda haber otras transformaciones mejores que Box-Cox en este caso.

In [ ]:
# Crear el Q-Q plot para la variable 'RS'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'RS'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['RS'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'RS' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'RS_boxcox'
plt.subplot(1, 2, 2)
stats.probplot(RS_boxcox, dist="norm", plot=plt)
plt.title("Q-Q plot de 'RS' (Transformación Box-Cox)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

En cuanto a normalidad, parece mejorar un poco, pero tampoco logra aproximar completamente por una distribución normal.

Por ello, vamos a probar ahora con una transformación logarítmica:

In [ ]:
# Aplica la transformación logarítmica
RS_log = np.log(df_concatenado_diario['RS'])

In [ ]:
# Crear el gráfico de distribución para la variable 'RS'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'RS'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['RS'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'RS' (Original)", fontsize=16)
plt.xlabel("Valores de 'RS'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'RS_log'
plt.subplot(1, 2, 2)
sns.histplot(RS_log, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'RS' (Transformación logarítmica)", fontsize=16)
plt.xlabel("Valores de 'RS' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

En este caso, se ha conseguido reducir considerablemente el rango.

In [ ]:
# Crear el Q-Q plot para la variable 'RS'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'RS'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['RS'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'RS' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'RS_log'
plt.subplot(1, 2, 2)
stats.probplot(RS_log, dist="norm", plot=plt)
plt.title("Q-Q plot de 'RS' (Transformación logarítmica)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

No obstante, no parece aproximar de forma adecuada por una distribución normal.

Por último, vamos a probar con una transformación de raíz cuadrada:

In [ ]:
# Aplica la transformación de raíz cuadrada
RS_sqrt = np.sqrt(df_concatenado_diario['RS'])

In [ ]:
# Crear el gráfico de distribución para la variable 'RS'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'RS'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['RS'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'RS' (Original)", fontsize=16)
plt.xlabel("Valores de 'RS'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'RS_sqrt'
plt.subplot(1, 2, 2)
sns.histplot(RS_sqrt, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'RS' (Transformación de raíz cuadrada)", fontsize=16)
plt.xlabel("Valores de 'RS' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Esta transformación logra disminuir el rango, aunque no tanto como la transformación logarítimca. Sin embargo, con la transformación de raíz cuadrada se logra una distribución más simétrica y más cercana a la normal, como se puede observar con el siguiente Q-Q plot:

In [ ]:
# Crear el Q-Q plot para la variable 'RS'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'RS'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['RS'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'RS' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'RS_sqrt'
plt.subplot(1, 2, 2)
stats.probplot(RS_sqrt, dist="norm", plot=plt)
plt.title("Q-Q plot de 'RS' (Transformación de raíz cuadrada)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Por lo tanto, finalmente nos quedamos con la transformación de raíz cuadrada.

In [ ]:
df_concatenado_diario_transformed['RS'] = RS_sqrt

### DD

La media de `DD` es de $191.505376$ y la desviación estándar es de $103.012220$, lo que indica una dispersión considerable de los datos. El rango de los valores va desde un valor mínimo de $0.005307$ hasta un máximo de $359.815207$.

Además, el coeficiente de simetría para la variable `DD` es de $B_1 = -0.403 < 0$, lo que indica una asimetría negativa moderada. Esto significa que la distribución tiene una mayor concentración de valores en el lado derecho de la media.

In [ ]:
# Calcular el coeficiente de simetría de la variable 'DD'
coeficiente_simetria = df_concatenado_diario['DD'].skew()
print(f"Coeficiente de simetría para la variable 'DD': {coeficiente_simetria}")

Observamos en el histograma que aparecen dos máximos. Esto sugiere que los datos podrían estar formados por dos poblaciones o un fenómeno que genera dos comportamientos distintos en los valores.

In [ ]:
# Crear el gráfico de distribución
plt.figure(figsize=(10, 6))
sns.histplot(df_concatenado_diario['DD'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'DD'", fontsize=16)
plt.xlabel("Valores de 'DD'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Esto puede ser más complicado de transformar de forma eficiente, por lo que vamos a ir probando cada una de las tres transformaciones consideradas, hasta encontrar la que mejor funcione.

Comenzamos con la transformación de raíz cuadrada:

In [ ]:
# Aplica la transformación de raíz cuadrada
DD_sqrt = np.sqrt(df_concatenado_diario['DD'])

In [ ]:
# Crear el gráfico de distribución para la variable 'DD'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'DD'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['DD'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'DD' (Original)", fontsize=16)
plt.xlabel("Valores de 'DD'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'DD_sqrt'
plt.subplot(1, 2, 2)
sns.histplot(DD_sqrt, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'DD' (Transformación de raíz cuadrada)", fontsize=16)
plt.xlabel("Valores de 'DD' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Observamos que se logra reducir bastante el rango de la variable, lo cual puede ser un punto a favor para la transformación de raíz cuadrada.

In [ ]:
# Crear el Q-Q plot para la variable 'DD'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'DD'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['DD'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'DD' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'DD_sqrt'
plt.subplot(1, 2, 2)
stats.probplot(DD_sqrt, dist="norm", plot=plt)
plt.title("Q-Q plot de 'DD' (Transformación de raíz cuadrada)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

No obstante, no logra aproximar la distribución de la variable por una normal de forma consistente.

Pasamos ahora con la transformación logarítmica:

In [ ]:
# Aplica la transformación logarítmica
DD_log = np.log(df_concatenado_diario['DD'])

In [ ]:
# Crear el gráfico de distribución para la variable 'DD'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'DD'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['DD'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'DD' (Original)", fontsize=16)
plt.xlabel("Valores de 'DD'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'DD_log'
plt.subplot(1, 2, 2)
sns.histplot(DD_log, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'DD' (Transformación logarítmica)", fontsize=16)
plt.xlabel("Valores de 'DD' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Claramente en este caso se obtiene una distribución muy asimétrica, además de tomar valores negativos, por lo que no parece la logarítmica no parece la transformación más adecuada.

Por último, vamos a probar con la transformación Box-Cox:

In [ ]:
# Aplica la transformación de Box-Cox
DD_boxcox, lambda_DD = boxcox(df_concatenado_diario['DD'])

In [ ]:
# Crear el gráfico de distribución para la variable 'DD'
plt.figure(figsize=(14, 8))

# Histograma de la variable original 'DD'
plt.subplot(1, 2, 1)
sns.histplot(df_concatenado_diario['DD'], kde=True, color='skyblue', bins=30)
plt.title("Distribución de la variable 'DD' (Original)", fontsize=16)
plt.xlabel("Valores de 'DD'", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Histograma de la variable transformada 'DD_boxcox'
plt.subplot(1, 2, 2)
sns.histplot(DD_boxcox, kde=True, color='lightcoral', bins=30)
plt.title("Distribución de la variable 'DD' (Transformación Box-Cox)", fontsize=16)
plt.xlabel("Valores de 'DD' Transformada", fontsize=14)
plt.ylabel("Frecuencia", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

El rango de la variable no consigue reducirse significativamente.

In [ ]:
# Crear el Q-Q plot para la variable 'DD'
plt.figure(figsize=(14, 8))

# Q-Q plot para la variable original 'DD'
plt.subplot(1, 2, 1)
stats.probplot(df_concatenado_diario['DD'], dist="norm", plot=plt)
plt.title("Q-Q plot de 'DD' (Original)", fontsize=16)

# Q-Q plot para la variable transformada 'DD_boxcox'
plt.subplot(1, 2, 2)
stats.probplot(DD_boxcox, dist="norm", plot=plt)
plt.title("Q-Q plot de 'DD' (Transformación Box-Cox)", fontsize=16)

# Mostrar los gráficos
plt.tight_layout()
plt.show()

La transformación Box-Cox tampoco consigue aproximar la distribución por una normal.

Por ello, como conclusión parece que la mejor transformación es la raíz cuadrada, ya que por lo menos consigue reducir considerablemente el rango de la variable `DD`.

In [ ]:
df_concatenado_diario_transformed['DD'] = DD_sqrt

## Modelos

#### Métricas

Para la evaluación de los modelos, usaremos dos métricas diferentes. Sea $X_t$ el valor real y $hat X_t$ el valor predicho, y sea $e_t = X_t - \hat X_t$ el error que cometemos con dicha predicción. Entonces, a la hora de buscar un error global, se pueden usar las siguientes métricas:

- Mean Absolute Percent Error (MAPE): mide el error promedio en términos relativos como un porcentaje del valor real. Al estar expresado como un porcentaje, es fácil de interpretar. Además, dicho porcentaje indica en qué medida la predicción se desvía del valor real. Sin embargo, este error es sensible a valores cercanos a cero, ya que al dividir entre un número cercano a cero, se amplifica el error. Se define como:

$$MAPE = \frac{1}{H}\sum_{t=1}^H\frac{100|e_t|}{X_t}$$

- Root Mean Squared Error (RMSE): mide el promedio del error en términos absolutos, ponderando en mayor medida los errores grandes debido al cuadrado de los términos $e_t$. Por esta razón, si hay outliers en los datos, puede estar sesgado. Sin embargo, al usar la raíz cuadrada, la penalización de los errores grandes es algo más severa que en el caso de MSE (dado que $RMSE = \sqrt{MSE}$). Como tiene las mismas unidades que la variable estudiada, se puede comparar directamente con ella. No obstante, es más difícil de interpretar en comparación con el MAPE, especialmente cuando la variable estudiada tiene una escala grande. Se define como:

$$RMSE = \sqrt{\frac{1}{H}\sum_{t=1}^He_t^2}$$

In [ ]:
def mean_absolute_percent_error(y_true, y_pred):
    # Evitar división por cero
    epsilon = 1e-8
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + epsilon)) * 100)
    if mape > 100 or mape == np.inf:
        return 100
    else:
        return mape

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

### Solo con la variable NOX

Para empezar, vamos a hacer algunas predicciones iniciales con modelos simples que se entrenan y hacen las predicciones únicamente usando la variable `NOX`. Inicialmente, emplearemos un modelo baseline muy simple, que nos permitirá obtener una primera aproximación de los errores y evaluar el comportamiento general de los datos. Este enfoque servirá como punto de referencia para comparar el desempeño de modelos más avanzados.

Además, utilizaremos un modelo con ventana deslizante, en el que se incorpora estructura temporal para capturar patrones en los datos de `NOX`. Este modelo, aunque todavía sencillo, permite integrar información de períodos anteriores de la variable `NOX`, además de ofrecer unos resultados más robustos para nuestras predicciones iniciales.

#### Modelo: baseLine

En primer lugar, vamos a probar como modelo baseline el modelo `ForecasterEquivalentDate`, también conocido como *Seasonal Naive Forecasting* que simplemente devuelve el valor observado en el mismo periodo de la temporada anterior. Dado que hemos observado cierta estacionalidad anual, se asume que el comportamiento de la serie en el mismo período de tiempo en el año anterior es una buena aproximación para el futuro. De esta forma, la predicción obtenida mediante este modelo para el valor de la variable `NOX` para un día será simplemente el valor del mismo día en el año anterior. Por tanto, es evidente que este modelo utiliza únicamente la variable `NOX` para realizar las predicciones

Definimos el modelo y lo entrenamos con los datos de entrenamiento. Los parámetros de este modelo son los siguientes:

- `offset`: Define el desplazamiento temporal utilizado para identificar las fechas equivalentes. En este caso, `pd.DateOffset(days=365)` significa que se buscarán datos del mismo día del año anterior.
- `n_offset`: Especifica el número de desplazamientos que se considerarán para hacer las predicciones. Por ejemplo, en nuestro caso, `n_offsets = 1` indica que el modelo utilizará únicamente el valor del año inmediatamente anterior como predicción.
- `agg_func`: Se refiere a la función usada para agregar los valores de las fechas equivalentes cuando `n_offset` es mayor que $1$. Como en nuestro caso `n_offsets = 1`, esta función no se usará.

In [ ]:
baseLine = ForecasterEquivalentDate(
                 offset    = pd.DateOffset(days=365),
                 n_offsets = 1,
                 agg_func  = 'mean',
             )

A continuación, entrenamos el modelo con la serie temporal de train formada únicamente por la variable `NOX`. Además, se generan predicciones para el mismo período que los datos de test. Por último, se filtran los datos para restringir la visualización a partir de una fecha específica, facilitando la intrepretación de los resultados.

In [ ]:
baseLine.fit(y = train_NOX)
baseLine_pred = baseLine.predict(steps = len(test_NOX))

# Filtrar los datos para mostrar solo a partir de 2022
train_NOX_filtered = train_NOX[train_NOX.index >= '2022-01-01']
test_NOX_filtered = test_NOX[test_NOX.index >= '2022-01-01']
baseLine_pred_filtered = baseLine_pred[baseLine_pred.index >= '2022-01-01']

En el siguiente código se generan las predicciones de `NOX` en los 7 días futuros, posteriores a los datos de nuestro dataset, usando el modelo `baseLine`. Además, se visualizan los resultados en un gráfico en el que se muestran las predicciones del conjunto de test junto a los valores reales de test, además de las predicciones para los 7 días futuros.

In [ ]:
# Crear índice de fechas para los próximos 7 días
future_dates = pd.date_range(start=test_NOX.index[-1] + pd.Timedelta(days=1), periods=7, freq='D')

# Crear un DataFrame para representar las nuevas fechas
future_df = pd.DataFrame(index=future_dates)

# Hacer predicciones para los próximos 7 días con ForecasterEquivalentDate
baseLine_future_pred = baseLine.predict(steps=len(future_dates))

# Crear un DataFrame con las predicciones futuras
future_predictions_df = pd.DataFrame({'Fecha': future_dates, 'predicciones': baseLine_future_pred})
future_predictions_df.set_index('Fecha', inplace=True)

# Mostrar las predicciones futuras
print(future_predictions_df)

# Graficar los resultados incluyendo las predicciones futuras
fig, ax = plt.subplots(figsize=(6, 3))
train_NOX_filtered.plot(ax=ax, label='train')
test_NOX_filtered.plot(ax=ax, label='test')
baseLine_pred_filtered.plot(ax=ax, label='baseline predictions')
future_predictions_df['predicciones'].plot(ax=ax, label='future predictions', color='red')
ax.legend()
plt.title('Baseline Predictions Including Future Dates')
plt.show()

En el gráfico se puede observar que las predicciones obtenidas por el modelo `baseLine` en el conjunto de test, si bien no se alejan de una forma exagerada de los valores reales, no logran captar con precisión las variaciones presentes en los datos originales. Esto es esperable dado que el modelo `baseLine` calcula las predicciones basándose únicamente en el valor de la variable `NOX` en el año anterior, lo que lo hace un modelo muy simple e incapaz de capturar fluctuaciones más complejas en los datos.

Además, los predicciones futuras para los próximos 7 días muestran una continuidad en la tendencia de los datos, dando una primera estimación de los valores esperados, aunque con claras limitaciones, especialmente en situaciones donde el comportamiento de los datos pueda ser más dinámico.

##### Error Backtesting

Para evaluar la robustez del modelo, utilizamos el método backtesting, que consiste en divir los datos históricos en varias ventanas de tiempo. En cada iteración, se entrena el modelo en un subconjunto inicial, y se evalúa su desempeño en un subconjunto consecutivo. Este procedimimiento se repite, desplazando las ventanas hacia adelante en el tiempo.

Gracias a este enfoque, se logran métricas de error que reflejan de una forma más realista el desempeño del modelo en datos que no ha visto durante su entrenamiento. En este caso, las métricas usadas para el backtesting serán el MAPE y el RMSE.

In [ ]:
cv = TimeSeriesFold(
    steps = 7,
    refit = False,
    initial_train_size = 1000
)

baseLine_backtesting_error, predictions_back = backtesting_forecaster(
    forecaster = baseLine,
    y = train_NOX,
    cv = cv,
    metric = [mean_absolute_percent_error, root_mean_squared_error],
    n_jobs = 'auto',
    verbose = True,
    show_progress = True
)

In [ ]:
print(baseLine_backtesting_error)

Tras realizar el backtesting, observamos que las métricas de error obtenidas son las siguientes:

- $MAPE = 49.114614\%$

    Este valor indica que, en promedio, las predicciones del modelo tienen un error relativo del $49.11\%$ respecto de los valores reales. Un MAPE alto como este ($49.11$ sobre $100$) sugiere que el modelo tiene dificultades para ajustarse con precisión a la serie temporal. Esto es esperable, debido a la gran simplicidad del modelo `baseLine`.

- $RMSE = 8.479244$

    Para poder interpretar este valor, debemos conocer algunos estadísticos para la variable `NOX`. Dado que el valor mínimo de esta variable es $3.25$ y el máximo es $60$, un $RMSE = 8.479244$ es un valor relativamente alto en comparación con el rango de la variable. De hecho, en términos relativos, el $RMSE$ es aproximadamente el $14\%$ del valor máximo de `NOX`, lo cual indica que las predicciones del modelo en promedio se están desviando bastante de los valores reales, más específicamente se desvían un $14\%$ del máximo de la variable.

    Por otro lado, la desviación estándar ($7.493708$) de `NOX` es bastante alta, lo cual indica que hay una dispersión considerable en los datos. De esta forma, aunque el modelo no está produciendo muy buenas predicciones, es importante tener en cuenta que las predicciones no están tan desviadas del comportamiento esperado. La alta variabilidad puede atenuar la interpretación del RMSE como un error excesivo, por lo que en nuestron caso, un error absoluto de $8.48$ podría ser ligeramente más aceptable.

In [ ]:
df_concatenado_diario['NOX'].describe()

##### Error Test

Ahora, calculamos el error RMSE y MAPE, pero esta vez sobre las predicciones que calculamos anteriormente para el conjunto de test.

In [ ]:
baseLine_test_error_rmse = root_mean_squared_error(
                y_true = test_NOX,
                y_pred = baseLine_pred
            )

baseLine_test_error_mape = mean_absolute_percent_error(
                y_true = test_NOX,
                y_pred = baseLine_pred
            )

print(f"Error RMSE: {baseLine_test_error_rmse}")
print(f"Error MAPE: {baseLine_test_error_mape:.2f}%\n")

Observamos que en este caso, el error obtenido con ambas métricas es claramente menor que en el backtesting. Por un lado, el $RMSE = 3.0975$ indica que el modelo se desvió en aproximadamente $3.0975$ unidades de `NOX` en el conjunto de prueba. Por otro lado, el $MAPE = 19.49\%$ indica que en promedio, las predicciones están desviadas en un $19.49\%$ respecto a los valores reales.

Una posible razón por la que se podría dar esta diferencia tan significativa entre los errores de test y los de backtesting es que los datos seleccionados en el conjunto de test sean más fáciles o representativos del comportamiento general del modelo, mientras que en los datos de backtesting, entre todas las ventanas de tiempo seleccionadas, podría haber datos más complejos o outliers, haciendo que el modelo tuviera más dificultades para ajustarse.

#### Modelo: RandomForestRegressor con ventana deslizante

El siguiente modelo que vamos a utilizar va a ser un RandomForestRegressor con ventana deslizante, combinando los modelos de árboles de decisión con un enfoque de series temporales. El modelo RandomForestRegressor está formado por un conjunto de árboles de decisión entrenados sobre subconjuntos aleatorios de los datos.

Por otro lado, la ventana deslizante es una técnica que utiliza ventanas de tiempo en las cuales se van utilizando los datos pasados (valores en $t-1$, $t-2$, $t-3$, etc) para predecir el valor en $t+1$. Así, usaremos la siguiente función para crear las variables lags:

In [ ]:
def create_lagged_features(data, lags, horizons, target_column, dropna=False):
    """
    Crea variables lags y variables para el horizonte temporal a partir de una columna objetivo. 
    
    Parámetros:
    - data: pd.DataFrame - Dataframe original con la columna objetivo de la predicción. 
    - lags: list[int] - Lista de lags a introducir en el dataframe (e.g., [1, 2, 3] for t-1, t-2, t-3).
    - horizons: list[int] - Lista de pasos en el horizonte temporal de predicción. lis (e.g., [1, 2, 3] for t+1, t+2, t+3).
    - target_column: str - Nombre de la columna que contiene la serie a predecir. 
    
    Devuelve:
    - pd.DataFrame - Dataframe con las variables lags y las variables del horizonte temporal 
    """
    
    df = data.copy()
    
    # Creación de las variables lags
    for lag in lags:
    #    df[f'{target_column}_lag_{lag}'] = df[target_column].shift(lag)
        df.insert(loc=0, column=f'{target_column}_lag_{lag}',value=df[target_column].shift(lag))
    
    # Creación de las variables en el horizonte de predicción. 
    for horizon in horizons:
        df[f'{target_column}_horizon_{horizon+1}'] = df[target_column].shift(-horizon)
    
    # Se eliminan los valores ausentes creados. 
    if dropna:
        df = df.dropna()
    
    
    return df

Seleccionamos la variable `NOX` del dataset:

In [ ]:
NOX = df_concatenado_diario['NOX']

NOX_df = pd.DataFrame(NOX, index=NOX.index)
NOX_df

A continuación se crean las variables lags, usando la función anterior. Dado que hemos observado que nuestra serie tiene una cierta estacionariedad semanal, y que además el objetivo de la práctica es predecir los próximos 7 días de la serie para la variable `NOX`, vamos a probar a tomar 7 variables lags a partir de dicha variable.

In [ ]:
df_NOX_lagged = create_lagged_features(NOX_df,lags=range(1,8),horizons=range(1,1),target_column='NOX')
df_NOX_lagged

Al obtener el dataset con las variables lags, observamos que, como es de esperar, las primeras 7 filas contienen valores nulos (resultado de la creación de la ventana con las variables lags). Por ello, vamos a eliminar estas filas:

In [ ]:
# Eliminar las primeras 7 filas (contienen valores NaN)
df_NOX_lagged = df_NOX_lagged.iloc[7:]

Ahora, con el dataset resultante, que contiene la variable `NOX` y sus correspondientes variables lags, vamos a separar los datos en un conjunto de train y otro de test:

In [ ]:
porcentaje_entrenamiento = 0.9
test_steps = int(len(df_NOX_lagged) * porcentaje_entrenamiento)
df_NOX_lagged_train = df_NOX_lagged[:test_steps]
df_NOX_lagged_test = df_NOX_lagged[test_steps:]

print(f'Con el porcentaje de %.2f tenemos:' %porcentaje_entrenamiento)
print(f'Tamaño del conjunto de training es %i' %len(df_NOX_lagged_train))
print(f'Tamaño del conjunto de test es %i' %len(df_NOX_lagged_test))
print(f'El conjunto de training va de {min(df_NOX_lagged_train.index)} y {max(df_NOX_lagged_train.index)}')
print(f'El conjunto de test va de {min(df_NOX_lagged_test.index)} y {max(df_NOX_lagged_test.index)}')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
df_NOX_lagged_train['NOX'].plot(ax=ax, label='train')
df_NOX_lagged_test['NOX'].plot(ax=ax, label='test')
ax.legend(labels=['Train', 'Test'], loc='best')
ax.set_xlabel('Año')
ax.set_ylabel('NOX')
ax.set_title('NOX mensual')
fig.tight_layout()
plt.show()

Separamos las variables independientes de las dependientes en ambos conjuntos. 

In [ ]:
X_NOX_lagged_train = df_NOX_lagged_train.drop(columns = 'NOX')
y_NOX_lagged_train = df_NOX_lagged_train['NOX']

X_NOX_lagged_test = df_NOX_lagged_test.drop(columns='NOX')
y_NOX_lagged_test = df_NOX_lagged_test['NOX']

Ahora que tenemos preparado el DataFrame, comenzamos a crear el modelo predictivo. Primero creamos una instancia del modelo que queremos entrenar, que en este caso, va a ser un random forest para regresión. Para buscar sus mejores parámetros, usamos `RandomizedSearchCV` que realiza una búsqueda aleatoria, en lugar de probar todas las combinaciones posibles entre los parámetros indicados. Esto es conveniente para modelos más complejos o con un grid con demasiados hiperparámetros, ya que probar todas las combinaciones posibles puede ser muy costoso computacionalmente.

Algunos de los parámetros de este modelo son:

- `n_estimators`: especifica el número de árboles que compondrán el modelo.
- `max_depth`: define la profundidad máxima de los árboles, es decir, el número máximo de divisiones que cada árbol puede hacer. Si toma un valor demasiado alto, puede llevar a overfitting.
- `min_samples_split`: establece el número mínimo de muestras que debe haber en un nodo para que se realice una división. Si se establece un valor bajo, el modelo será más propenso al sobreajuste.
- `min_samples_leaf`: especifica el número mínimo de muestras que debe tener una hoja para que sea válida. Si se establece un valor bajo, el modelo puede crear hojas con pocos datos, lo que puede hacer que el modelo se sobreajuste al ruido en los datos de entrenamiento.
- `max_features`: determina el número de características a considerar al dividir un nodo en los árboles, por ejemplo, si es `"sqrt"` se selecciona la raíz cuadrada del total de características, y si es `"log2"` se selecciona el logaritmo en base $2$ del total de características.
- `bootstrap`: indica si se utiliza el muestreo con o sin reemplazo al construir los árboles.

In [ ]:
# Definimos los rangos de los parámetros
param_dist = {
    "n_estimators": [100, 200, 300, 500],  # Número de árboles
    "max_depth": [2, 3],  # Profundidad máxima de los árboles
    "min_samples_split": [5, 10, 20],  # Mínimas muestras para dividir
    "min_samples_leaf": [4, 5, 10],  # Mínimas muestras en hoja
    "max_features": ["sqrt", "log2"],  # Selección de características
    "bootstrap": [True, False],  # Muestreo con o sin reemplazo
}

# Definimos el modelo base
rf_forecaster = RandomForestRegressor(random_state = semilla)

# Creamos un scoring personalizado con MAPE
mape = make_scorer(mean_absolute_percent_error, greater_is_better=False)

# Configuramos RandomizedSearchCV
random_cv = RandomizedSearchCV(
    estimator=rf_forecaster,
    param_distributions=param_dist,
    n_iter=15,  # Número de iteraciones aleatorias
    scoring=mape,  # Métrica
    cv=5,  # Número de particiones para validación cruzada
    verbose=1,
    random_state=semilla
)

# Realizamos la búsqueda
random_cv.fit(X_NOX_lagged_train, y_NOX_lagged_train)

# Mejor modelo y parámetros
rf_forecaster_best = random_cv.best_estimator_
print("Best parameters:", random_cv.best_params_)
print("Best score:", random_cv.best_score_)

Para construir el modelo, procedemos con la función `.fit()` sobre el conjunto de entrenamiento. A continuación, realizamos las predicciones en el conjunto de test usando `.predict`.

In [ ]:
rf_fit = rf_forecaster_best.fit(X_NOX_lagged_train, y_NOX_lagged_train)

y_pred_test_rf = rf_fit.predict(X_NOX_lagged_test)
y_pred_test_rf = pd.DataFrame(y_pred_test_rf, index=X_NOX_lagged_test.index, columns=['predicciones'])

Ahora mostramos un gráfico con las predicciones y los valores reales de test. Se puede apreciar que con este modelo, las predicciones parecen algo mejores que con el modelo `baseLine` probado anteriormente.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

# Graficar los datos de entrenamiento, prueba y predicciones
df_NOX_lagged_train['NOX'].plot(ax=ax, label='Entrenamiento')
df_NOX_lagged_test['NOX'].plot(ax=ax, label='Prueba')
y_pred_test_rf.plot(ax=ax, color='green', alpha=0.7, label='Predicciones')

# Configurar el límite en el eje x para mostrar datos desde 2022-01-01
ax.set_xlim(pd.Timestamp('2022-01-01'), None)  # Límite inferior, sin límite superior

# Personalizar etiquetas y leyendas
ax.set_xlabel('Año')
ax.set_ylabel('NOX')
ax.set_title('NOX mensual')
ax.legend(labels=['Entrenamiento', 'Prueba', 'Predicciones'])

# Ajustar diseño
fig.tight_layout()
plt.show()

Calculamos ahora los errores obtenidos en el conjunto de test. Los errores obtenidos son:

- $MAPE = 27.00\%$

    El $MAPE$ mide el error absoluto relativo en porcentaje entre las predicciones y los valores reales. En este caso, el $MAPE$ es de $27.00\%$, lo que significa que las predicciones se desvían en promedio un $27\%$ respecto a los valores reales. Este valor es relativamente alto, lo que indica que las predicciones del modelo no son tan precisas.

- $RMSE = 1.9765$

    El $RMSE$ mide la raíz cuadrada de la media de los errores al cuadrado. En este caso, el $RMSE$ es de $1.98$. El rango de `NOX` va de $3.25$ a $60$, por lo que un $RMSE$ de $1.98$ es relativamente bajo en comparación con el rango de la variable. De hecho, este valor de $RMSE$ representa un $3.3\%$ del valor máximo de la variable. Este porcentaje también indica que, aunque el modelo tiene algunos errores, los errores no son excesivamente grandes en relación con el rango de los datos.

En resumen, aunque los errores no son extremadamente altos, hay espacio para mejorar las predicciones, especialmente en lo que respecta al $MAPE$.

In [ ]:
mape = mean_absolute_percent_error(y_NOX_lagged_test, y_pred_test_rf)
rmse = root_mean_squared_error(y_NOX_lagged_test, y_pred_test_rf)

print(f'MAPE: {mape}%')
print('RMSE:', rmse)

Por último, en la siguiente celda se calculan las predicciones para los 7 días futuros, posteriores a los datos de nuestro dataset.

In [ ]:
# Inicializamos un DataFrame para almacenar predicciones futuras
future_predictions = []

# Copiar las últimas filas de X_NOX_lagged_test (las usaremos como base)
X_future = X_NOX_lagged_test.iloc[-1:].copy()

# Hacer predicciones para los 7 días futuros
for i in range(7):
    # Predecir el próximo día
    next_pred = rf_forecaster_best.predict(X_future)[0]

    # Guardamos la predicción
    future_predictions.append(next_pred)

    # Actualizar las variables lagged
    for lag in range(4, 1, -1):  # Desde NOX_lag_4 hasta NOX_lag_2
        X_future[f'NOX_lag_{lag}'] = X_future[f'NOX_lag_{lag-1}']
    X_future['NOX_lag_1'] = next_pred  # Actualizamos NOX_lag_1 con la nueva predicción

# Crear DataFrame final con las fechas y predicciones
future_dates = pd.date_range(start=X_NOX_lagged_test.index[-1] + pd.Timedelta(days=1), periods=7, freq='D')
future_predictions_df = pd.DataFrame({'Fecha': future_dates, 'predicciones': future_predictions})
future_predictions_df.set_index('Fecha', inplace=True)

# Mostrar predicciones
print(future_predictions_df)

### Predicción recursiva

Preparamos los datasets para tener tanto sin transformar como transformados con solo variables del aire.

In [ ]:
df_concatenado_aire_diario = df_concatenado_diario.drop(['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'], axis=1)
df_concatenado_aire_diario_transformed = df_concatenado_diario_transformed.drop(['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'], axis=1)

Usamos la función `create_lagged_features` para crear las variables lagged de las series temporales.

In [ ]:
def create_lagged_features(data, lags, horizons, target_column, dropna=False):
    """
    Crea variables lags y variables para el horizonte temporal a partir de una columna objetivo.

    Parámetros:
    - data: pd.DataFrame - Dataframe original con la columna objetivo de la predicción.
    - lags: list[int] - Lista de lags a introducir en el dataframe (e.g., [1, 2, 3] for t-1, t-2, t-3).
    - horizons: list[int] - Lista de pasos en el horizonte temporal de predicción. lis (e.g., [1, 2, 3] for t+1, t+2, t+3).
    - target_column: str - Nombre de la columna que contiene la serie a predecir.

    Devuelve:
    - pd.DataFrame - Dataframe con las variables lags y las variables del horizonte temporal
    """

    df = data.copy()

    # Creación de las variables lags
    for lag in lags:
    #    df[f'{target_column}_lag_{lag}'] = df[target_column].shift(lag)
        df.insert(loc=0, column=f'{target_column}_lag_{lag}',value=df[target_column].shift(lag))

    # Creación de las variables en el horizonte de predicción.
    for horizon in horizons:
        df[f'{target_column}_horizon_{horizon+1}'] = df[target_column].shift(-horizon)

    # Se eliminan los valores ausentes creados.
    if dropna:
        df = df.dropna()


    return df

Para poder facilitar la búsqueda de hiperparámetros hemos definido la función `process_variable` que para cada una de las variables y cada uno de las tres técnicas obtenga el mejor modelo optimizado. Lo guardamos de forma que podamos cargarlo en el futuro.

In [ ]:
def process_variable(variable_name, df, test_size, param_grid, regressor, model_dir, train_mode=True):
    """
    Procesa una variable para entrenamiento, optimización y predicción.

    Args:
        variable_name (str): Nombre de la variable objetivo.
        df (pd.DataFrame): DataFrame con las series temporales.
        test_size (int): Tamaño del conjunto de prueba.
        param_grid (dict): Grid de hiperparámetros para optimización.
        regressor: Regresor a usar en el modelo.
        model_dir (str): Directorio donde guardar/cargar los modelos.
        train_mode (bool): Si es True, entrena el modelo; si es False, utiliza un modelo preentrenado.

    Returns:
        pd.DataFrame: Predicciones finales.
        float: MAPE del modelo.
        float: RMSE del modelo.
    """
    # Preparación de datos
    test_restante = test_size
    train = df[variable_name].to_frame()
    train_actual = train[:-test_restante]

    # Crear el directorio de modelos si no existe
    os.makedirs(model_dir, exist_ok=True)
    model_filename = os.path.join(model_dir, f"model_{variable_name}.joblib")

    if train_mode:
        # Instanciar el ForecasterAutoreg con el regresor especificado
        forecaster = ForecasterRecursive(
            regressor=regressor,
            lags=range(1, 8)  # Lags de 1 a 7
        )

        # Realizar la búsqueda de hiperparámetros
        cv = TimeSeriesFold(
            steps=7,
            initial_train_size=520,
            refit=True
        )

        results = grid_search_forecaster(
            forecaster=forecaster,
            y=train[variable_name],
            param_grid=param_grid,
            lags_grid=None,
            cv=cv,
            metric=[mean_absolute_percent_error, root_mean_squared_error],
            n_jobs='auto',
            verbose=False,
            show_progress=True
        )

        best_params = results.loc[results['mean_absolute_percent_error'].idxmin()]
        print(f"Mejores hiperparámetros para {variable_name}: {best_params}")
        model_params = best_params['params']

        # Actualizar el regresor con los mejores parámetros
        forecaster.regressor = regressor.__class__(**model_params)

        # Guardar el modelo entrenado
        save_forecaster(
            forecaster=forecaster,
            file_name=model_filename,
            save_custom_functions=True,
            verbose=True
        )
        print(f"Modelo guardado como {model_filename}")
    else:
        if os.path.exists(model_filename):
            forecaster = load_forecaster(model_filename)
            print(f"Modelo cargado desde {model_filename}")
        else:
            raise FileNotFoundError(f"El modelo preentrenado {model_filename} no se encontró.")

    # Predicciones
    final_predictions = pd.DataFrame()

    while test_restante > 0:
        lagged_features = create_lagged_features(
            train_actual, lags=range(1, 8), horizons=range(1, 1), target_column=variable_name
        )

        lagged_features.dropna(inplace=True)

        X_train = lagged_features.drop(columns=variable_name)
        y_train = lagged_features[variable_name]
        forecaster.fit(y=y_train, exog=None)

        if test_restante >= 7:
            predictions = forecaster.predict(steps=7)
        else:
            predictions = forecaster.predict(steps=test_restante)

        final_predictions = pd.concat([final_predictions, predictions], axis=0)
        test_restante -= min(7, test_restante)
        train_actual = train[:-test_restante] if test_restante > 0 else train

    # Evaluación
    print(f"Predicciones para {variable_name}:\n{final_predictions}")
    test_actual = train[-test_size:]
    mape = mean_absolute_percent_error(test_actual, final_predictions)
    rmse = root_mean_squared_error(test_actual, final_predictions)

    # Graficar resultados
    fig, ax = plt.subplots(figsize=(6, 3))
    test_actual.plot(ax=ax, label='Test (real)')
    final_predictions.plot(ax=ax, label='Predicciones')
    ax.legend()
    plt.title(f'Predicciones {variable_name}')
    plt.show()

    print(f"MAPE para {variable_name}: {mape:.2%}")
    print(f"RMSE para {variable_name}: {rmse:.4f}")

    return final_predictions, mape, rmse

def process_variable_sin_mape(variable_name, df, test_size, param_grid, regressor, model_dir, train_mode=True):
    """
    Procesa una variable para entrenamiento, optimización y predicción.

    Args:
        variable_name (str): Nombre de la variable objetivo.
        df (pd.DataFrame): DataFrame con las series temporales.
        test_size (int): Tamaño del conjunto de prueba.
        param_grid (dict): Grid de hiperparámetros para optimización.
        regressor: Regresor a usar en el modelo.
        model_dir (str): Directorio donde guardar/cargar los modelos.
        train_mode (bool): Si es True, entrena el modelo; si es False, utiliza un modelo preentrenado.

    Returns:
        pd.DataFrame: Predicciones finales.
        float: MAPE del modelo.
        float: RMSE del modelo.
    """
    # Preparación de datos
    test_restante = test_size
    train = df[variable_name].to_frame()
    train_actual = train[:-test_restante]

    # Crear el directorio de modelos si no existe
    os.makedirs(model_dir, exist_ok=True)
    model_filename = os.path.join(model_dir, f"model_{variable_name}.joblib")

    if train_mode:
        # Instanciar el ForecasterAutoreg con el regresor especificado
        forecaster = ForecasterRecursive(
            regressor=regressor,
            lags=range(1, 8)  # Lags de 1 a 7
        )

        # Realizar la búsqueda de hiperparámetros
        cv = TimeSeriesFold(
            steps=7,
            initial_train_size=520,
            refit=True
        )

        results = grid_search_forecaster(
            forecaster=forecaster,
            y=train[variable_name],
            param_grid=param_grid,
            lags_grid=None,
            cv=cv,
            metric=[root_mean_squared_error],
            n_jobs='auto',
            verbose=False,
            show_progress=True
        )

        best_params = results.loc[results['mean_absolute_percent_error'].idxmin()]
        print(f"Mejores hiperparámetros para {variable_name}: {best_params}")
        model_params = best_params['params']

        # Actualizar el regresor con los mejores parámetros
        forecaster.regressor = regressor.__class__(**model_params)

        # Guardar el modelo entrenado
        save_forecaster(
            forecaster=forecaster,
            file_name=model_filename,
            save_custom_functions=True,
            verbose=True
        )
        print(f"Modelo guardado como {model_filename}")
    else:
        if os.path.exists(model_filename):
            forecaster = load_forecaster(model_filename)
            print(f"Modelo cargado desde {model_filename}")
        else:
            raise FileNotFoundError(f"El modelo preentrenado {model_filename} no se encontró.")

    # Predicciones
    final_predictions = pd.DataFrame()

    while test_restante > 0:
        lagged_features = create_lagged_features(
            train_actual, lags=range(1, 8), horizons=range(1, 1), target_column=variable_name
        )

        lagged_features.dropna(inplace=True)

        X_train = lagged_features.drop(columns=variable_name)
        y_train = lagged_features[variable_name]
        forecaster.fit(y=y_train, exog=None)

        if test_restante >= 7:
            predictions = forecaster.predict(steps=7)
        else:
            predictions = forecaster.predict(steps=test_restante)

        final_predictions = pd.concat([final_predictions, predictions], axis=0)
        test_restante -= min(7, test_restante)
        train_actual = train[:-test_restante] if test_restante > 0 else train

    # Evaluación
    print(f"Predicciones para {variable_name}:\n{final_predictions}")
    test_actual = train[-test_size:]
    #mape = mean_absolute_percent_error(test_actual, final_predictions)
    rmse = root_mean_squared_error(test_actual, final_predictions)

    # Graficar resultados
    fig, ax = plt.subplots(figsize=(6, 3))
    test_actual.plot(ax=ax, label='Test (real)')
    final_predictions.plot(ax=ax, label='Predicciones')
    ax.legend()
    plt.title(f'Predicciones {variable_name}')
    plt.show()

    #print(f"MAPE para {variable_name}: {mape:.2%}")
    print(f"RMSE para {variable_name}: {rmse:.4f}")

    return final_predictions, rmse



##### SIN TRANSFORMACION

###### Solo variables aire

Para cada una de las variables vamos a predecir sus valores con los modelos optimizados hasta las fechas de test para poder usarlas como variables endógenas.

In [ ]:
model_directory = "models/"
tipo_de_df = "sin_transformacion" #Cambiar con o sin transformación dependiendo

# Hiperparámetros comunes
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15]
}

# Lista de regresores
regressors = {
    "RandomForest": RandomForestRegressor(random_state=semilla),
    "GradientBoosting": GradientBoostingRegressor(random_state=semilla),
    "XGBoost": XGBRegressor(random_state=semilla)
}

# Proceso para cada variable y cada regresor
results = {}
variables = ["NO", "NO2", "SO2", "PM10", "O3"]

for var in variables:
    for reg_name, regressor in regressors.items():
        print(f"Procesando {var} con {reg_name}...")
        predictions, mape, rmse = process_variable(
            variable_name=var,
            df=df_concatenado_aire_diario,
            test_size=len(df_test),
            param_grid=param_grid,
            regressor=regressor,
            model_dir=model_directory + reg_name + "_" + tipo_de_df,
            train_mode=False  # Cambiar a False si deseas cargar modelos preentrenados
        )
        results[(var, reg_name)] = {"predictions": predictions, "mape": mape, "rmse": rmse}
        print(f"Resultados para {var} con {reg_name}: MAPE={mape:.2%}, RMSE={rmse:.4f}")



In [ ]:
predictions_df_test = {}

for reg_name in regressors.keys():
    print(f"\nProcesando predicciones para el modelo {reg_name}...\n")
    
    prediction_frames = []
    
    for var in variables:
        predictions_df = results[(var, reg_name)]['predictions']
        
        if 'pred' in predictions_df.columns:
            predictions_df = predictions_df[['pred']]  
            predictions_df.columns = [var] 
        else:
            print(f"Advertencia: No se encontró la columna 'pred' en las predicciones para {var} con {reg_name}.")
            continue
        
        prediction_frames.append(predictions_df)
    
    model_df = pd.concat(prediction_frames, axis=1)
    
    predictions_df_test[reg_name] = model_df
    
    print(f"Predicciones para el modelo {reg_name}:")
    print(model_df.head())  # Mostrar las primeras filas del DataFrame generado




In [ ]:
models = {
    "RandomForest": RandomForestRegressor(random_state=semilla),
    "GradientBoosting": GradientBoostingRegressor(random_state=semilla),
    "XGBoost": XGBRegressor(random_state=semilla)
}

param_grid = {
    'n_estimators': [100, 150, 250],
    'max_depth': [3, 5, 8],
    'min_samples_split': [2, 5, 10]
}

results_models = {}

for model_name, model in models.items():
    print(f"\nRealizando grid search para {model_name}...")

    forecaster = ForecasterRecursive(
        regressor=model,
        lags=7
    )

    cv = TimeSeriesFold(
        steps=7,
        initial_train_size=int(len(y_train_aire) * 0.35),
        refit=True
    )

    results_grid = grid_search_forecaster(
        forecaster=forecaster,
        y=y_train_aire,
        cv=cv,
        param_grid=param_grid,
        metric=[mean_absolute_percent_error, root_mean_squared_error],
        return_best=True,
        n_jobs=-1,
        exog=X_train_aire,  # Utilizar las variables exógenas como X_train_aire
        verbose=True
    )

    best_params = results_grid.loc[results_grid['mean_absolute_percent_error'].idxmin()]
    print(f"Mejores hiperparámetros para {model_name}: {best_params}")
    
    model_params = best_params['params']
    
    # Establecer los parámetros óptimos en el modelo
    model.set_params(**model_params)

    # Ajustar el modelo con los mejores parámetros
    forecaster.regressor = model

    model_filename = "models/" + model_name + "_sin_transformacion/" + "model_NOX.joblib"
    save_forecaster(
            forecaster=forecaster,
            file_name=model_filename,
            save_custom_functions=True,
            verbose=True
        )
    print(f"Modelo guardado como {model_filename}")

    forecaster.fit(y=y_train_aire, exog=X_train_aire)
    
    # Obtener las predicciones con el modelo ajustado
    predicciones = forecaster.predict(steps=len(X_test_aire), exog=predictions_df_test[model_name])

    # Graficar predicciones vs valores reales
    fig, ax = plt.subplots(figsize=(6, 3))
    y_test_aire.plot(ax=ax, label='Real')
    predicciones.plot(ax=ax, label='Predicciones')
    ax.legend()
    plt.title(f'Predicciones con el mejor modelo ({model_name})')
    plt.show()

    # Calcular métricas de error
    mape = mean_absolute_percentage_error(y_test_aire, predicciones)
    rmse = root_mean_squared_error(y_test_aire, predicciones)

    print(f"MAPE para {model_name}: {mape:.2%}")
    print(f"RMSE para {model_name}: {rmse:.4f}")

    results_models[model_name] = {
        "mape": mape,
        "rmse": rmse,
        "predicciones": predicciones
    }

# Mostrar resultados finales de todos los modelos
for model_name, model_results in results_models.items():
    print(f"\nResultados para {model_name}:")
    print(f"MAPE: {model_results['mape']:.2%}")
    print(f"RMSE: {model_results['rmse']:.4f}")


###### Predicción real sin transformación con variables de aire

Por falta de tiempo solo haremos predicciones reales usando variables de aire y sin transformación, sin embargo, el proceso es el mismo para las demás predicciones con transformación o sin ella, simplemente hay que cambiar el dataset y la ruta de los modelos.

Predicción real: GradientBoosting

In [ ]:
NO_dft = df_concatenado_diario["NO"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
NO_lagged = create_lagged_features(NO_dft, lags=range(1, 8), horizons=range(1,1), target_column='NO')
NO_lagged.dropna(inplace=True)

X_train = NO_lagged.drop(columns='NO')  # Características (lags)
y_train = NO_lagged['NO']               # Objetivo (NO)

model = load_forecaster('./models/GradientBoosting_sin_transformacion/model_NO.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_NO = model.predict(steps=7)
print(predictions_NO)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
NO_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_NO.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NO')
plt.show()

In [ ]:
NO2_dft = df_concatenado_diario["NO2"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
NO2_lagged = create_lagged_features(NO2_dft, lags=range(1, 8), horizons=range(1,1), target_column='NO2')
NO2_lagged.dropna(inplace=True)

X_train = NO2_lagged.drop(columns='NO2')  # Características (lags)
y_train = NO2_lagged['NO2']               # Objetivo (NO)

model = load_forecaster('./models/GradientBoosting_sin_transformacion/model_NO2.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_NO2 = model.predict(steps=7)
print(predictions_NO2)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
NO2_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_NO2.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NO2')
plt.show()

In [ ]:
SO2_dft = df_concatenado_diario["SO2"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
SO2_lagged = create_lagged_features(SO2_dft, lags=range(1, 8), horizons=range(1,1), target_column='SO2')
SO2_lagged.dropna(inplace=True)

X_train = SO2_lagged.drop(columns='SO2')  # Características (lags)
y_train = SO2_lagged['SO2']               # Objetivo (NO)

model = load_forecaster('./models/GradientBoosting_sin_transformacion/model_SO2.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_SO2 = model.predict(steps=7)
print(predictions_SO2)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
SO2_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_SO2.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones SO2')
plt.show()

In [ ]:
PM10_dft = df_concatenado_diario["PM10"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
PM10_lagged = create_lagged_features(PM10_dft, lags=range(1, 8), horizons=range(1,1), target_column='PM10')
PM10_lagged.dropna(inplace=True)

X_train = PM10_lagged.drop(columns='PM10')  # Características (lags)
y_train = PM10_lagged['PM10']               # Objetivo (NO)

model = load_forecaster('./models/GradientBoosting_sin_transformacion/model_PM10.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_PM10 = model.predict(steps=7)
print(predictions_PM10)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
PM10_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_PM10.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones PM10')
plt.show()

In [ ]:
O3_dft = df_concatenado_diario["O3"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
O3_lagged = create_lagged_features(O3_dft, lags=range(1, 8), horizons=range(1,1), target_column='O3')
O3_lagged.dropna(inplace=True)

X_train = O3_lagged.drop(columns='O3')  # Características (lags)
y_train = O3_lagged['O3']               # Objetivo (NO)

model = load_forecaster('./models/GradientBoosting_sin_transformacion/model_O3.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_O3 = model.predict(steps=7)
print(predictions_O3)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
O3_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_O3.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones O3')
plt.show()

In [ ]:
predictions_df = pd.DataFrame({
    'NO': predictions_NO,
    'NO2': predictions_NO2,
    'SO2': predictions_NO2,
    'PM10': predictions_PM10,
    'O3': predictions_O3
})

In [ ]:
predictions_df # Visualizamos el nuevo dataset predicho para los siguiente 7 días

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario.drop(columns = ['NOX','TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']

model = load_forecaster('./models/GradientBoosting_sin_transformacion/model_NOX.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_df_concatenado_aire_diario, exog=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = model.predict(steps=7, exog=predictions_df)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

Predicción real: RandomForest

In [ ]:
NO_dft = df_concatenado_diario["NO"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
NO_lagged = create_lagged_features(NO_dft, lags=range(1, 8), horizons=range(1,1), target_column='NO')
NO_lagged.dropna(inplace=True)

X_train = NO_lagged.drop(columns='NO')  # Características (lags)
y_train = NO_lagged['NO']               # Objetivo (NO)

model = load_forecaster('./models/RandomForest_sin_transformacion/model_NO.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_NO = model.predict(steps=7)
print(predictions_NO)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
NO_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_NO.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NO')
plt.show()

In [ ]:
NO2_dft = df_concatenado_diario["NO2"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
NO2_lagged = create_lagged_features(NO2_dft, lags=range(1, 8), horizons=range(1,1), target_column='NO2')
NO2_lagged.dropna(inplace=True)

X_train = NO2_lagged.drop(columns='NO2')  # Características (lags)
y_train = NO2_lagged['NO2']               # Objetivo (NO)

model = load_forecaster('./models/RandomForest_sin_transformacion/model_NO2.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_NO2 = model.predict(steps=7)
print(predictions_NO2)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
NO2_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_NO2.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NO2')
plt.show()

In [ ]:
SO2_dft = df_concatenado_diario["SO2"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
SO2_lagged = create_lagged_features(SO2_dft, lags=range(1, 8), horizons=range(1,1), target_column='SO2')
SO2_lagged.dropna(inplace=True)

X_train = SO2_lagged.drop(columns='SO2')  # Características (lags)
y_train = SO2_lagged['SO2']               # Objetivo (NO)

model = load_forecaster('./models/RandomForest_sin_transformacion/model_SO2.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_SO2 = model.predict(steps=7)
print(predictions_SO2)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
SO2_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_SO2.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones SO2')
plt.show()

In [ ]:
PM10_dft = df_concatenado_diario["PM10"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
PM10_lagged = create_lagged_features(PM10_dft, lags=range(1, 8), horizons=range(1,1), target_column='PM10')
PM10_lagged.dropna(inplace=True)

X_train = PM10_lagged.drop(columns='PM10')  # Características (lags)
y_train = PM10_lagged['PM10']               # Objetivo (NO)

model = load_forecaster('./models/RandomForest_sin_transformacion/model_PM10.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_PM10 = model.predict(steps=7)
print(predictions_PM10)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
PM10_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_PM10.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones PM10')
plt.show()

In [ ]:
O3_dft = df_concatenado_diario["O3"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
O3_lagged = create_lagged_features(O3_dft, lags=range(1, 8), horizons=range(1,1), target_column='O3')
O3_lagged.dropna(inplace=True)

X_train = O3_lagged.drop(columns='O3')  # Características (lags)
y_train = O3_lagged['O3']               # Objetivo (NO)

model = load_forecaster('./models/RandomForest_sin_transformacion/model_O3.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_O3 = model.predict(steps=7)
print(predictions_O3)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
O3_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_O3.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones O3')
plt.show()

In [ ]:
predictions_df = pd.DataFrame({
    'NO': predictions_NO,
    'NO2': predictions_NO2,
    'SO2': predictions_NO2,
    'PM10': predictions_PM10,
    'O3': predictions_O3
})

In [ ]:
predictions_df # Visualizamos el nuevo dataset predicho para los siguiente 7 días

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario.drop(columns = ['NOX','TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']

model = load_forecaster('./models/RandomForest_sin_transformacion/model_NOX.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_df_concatenado_aire_diario, exog=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = model.predict(steps=7, exog=predictions_df)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

Predicción real: XGBoost

In [ ]:
NO_dft = df_concatenado_diario["NO"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
NO_lagged = create_lagged_features(NO_dft, lags=range(1, 8), horizons=range(1,1), target_column='NO')
NO_lagged.dropna(inplace=True)

X_train = NO_lagged.drop(columns='NO')  # Características (lags)
y_train = NO_lagged['NO']               # Objetivo (NO)

model = load_forecaster('./models/XGBoost_sin_transformacion/model_NO.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_NO = model.predict(steps=7)
print(predictions_NO)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
NO_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_NO.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NO')
plt.show()

In [ ]:
NO2_dft = df_concatenado_diario["NO2"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
NO2_lagged = create_lagged_features(NO2_dft, lags=range(1, 8), horizons=range(1,1), target_column='NO2')
NO2_lagged.dropna(inplace=True)

X_train = NO2_lagged.drop(columns='NO2')  # Características (lags)
y_train = NO2_lagged['NO2']               # Objetivo (NO)

model = load_forecaster('./models/XGBoost_sin_transformacion/model_NO2.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_NO2 = model.predict(steps=7)
print(predictions_NO2)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
NO2_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_NO2.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NO2')
plt.show()

In [ ]:
SO2_dft = df_concatenado_diario["SO2"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
SO2_lagged = create_lagged_features(SO2_dft, lags=range(1, 8), horizons=range(1,1), target_column='SO2')
SO2_lagged.dropna(inplace=True)

X_train = SO2_lagged.drop(columns='SO2')  # Características (lags)
y_train = SO2_lagged['SO2']               # Objetivo (NO)

model = load_forecaster('./models/XGBoost_sin_transformacion/model_SO2.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_SO2 = model.predict(steps=7)
print(predictions_SO2)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
SO2_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_SO2.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones SO2')
plt.show()

In [ ]:
PM10_dft = df_concatenado_diario["PM10"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
PM10_lagged = create_lagged_features(PM10_dft, lags=range(1, 8), horizons=range(1,1), target_column='PM10')
PM10_lagged.dropna(inplace=True)

X_train = PM10_lagged.drop(columns='PM10')  # Características (lags)
y_train = PM10_lagged['PM10']               # Objetivo (NO)

model = load_forecaster('./models/XGBoost_sin_transformacion/model_PM10.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_PM10 = model.predict(steps=7)
print(predictions_PM10)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
PM10_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_PM10.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones PM10')
plt.show()

In [ ]:
O3_dft = df_concatenado_diario["O3"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
O3_lagged = create_lagged_features(O3_dft, lags=range(1, 8), horizons=range(1,1), target_column='O3')
O3_lagged.dropna(inplace=True)

X_train = O3_lagged.drop(columns='O3')  # Características (lags)
y_train = O3_lagged['O3']               # Objetivo (NO)

model = load_forecaster('./models/XGBoost_sin_transformacion/model_O3.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_O3 = model.predict(steps=7)
print(predictions_O3)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
O3_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_O3.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones O3')
plt.show()

In [ ]:
predictions_df = pd.DataFrame({
    'NO': predictions_NO,
    'NO2': predictions_NO2,
    'SO2': predictions_NO2,
    'PM10': predictions_PM10,
    'O3': predictions_O3
})

In [ ]:
predictions_df # Visualizamos el nuevo dataset predicho para los siguiente 7 días

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario.drop(columns = ['NOX','TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']

model = load_forecaster('./models/XGBoost_sin_transformacion/model_NOX.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_df_concatenado_aire_diario, exog=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = model.predict(steps=7, exog=predictions_df)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

###### Con todas las variables

Ahora vamos a hacer lo mismo pero con las variables que hemos hecho en la sección anterior pero esta vez con todas las variables meteorológicas.

In [ ]:
model_directory = "models/"
tipo_de_df = "sin_transformacion" #Cambiar con o sin transformación dependiendo

# Hiperparámetros comunes
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15]
}

# Lista de regresores
regressors = {
    "RandomForest": RandomForestRegressor(random_state=semilla),
    "GradientBoosting": GradientBoostingRegressor(random_state=semilla),
    "XGBoost": XGBRegressor(random_state=semilla)
}

# Proceso para cada variable y cada regresor
results = {}
variables = ["TMP", "HR", "DD", "PRB", "VV", "RS"]

for var in variables:
    for reg_name, regressor in regressors.items():
        print(f"Procesando {var} con {reg_name}...")
        predictions, mape, rmse = process_variable(
            variable_name=var,
            df=df_concatenado_diario,
            test_size=len(df_test),
            param_grid=param_grid,
            regressor=regressor,
            model_dir=model_directory + reg_name + "_" + tipo_de_df,
            train_mode=True  # Cambiar a False si deseas cargar modelos preentrenados
        )
        results[(var, reg_name)] = {"predictions": predictions, "mape": mape, "rmse": rmse}
        print(f"Resultados para {var} con {reg_name}: MAPE={mape:.2%}, RMSE={rmse:.4f}")

In [ ]:
model_directory = "models/"
tipo_de_df = "sin_transformacion" #Cambiar con o sin transformación dependiendo

# Hiperparámetros comunes
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15]
}

# Lista de regresores
regressors = {
    "RandomForest": RandomForestRegressor(random_state=semilla),
    "GradientBoosting": GradientBoostingRegressor(random_state=semilla),
    "XGBoost": XGBRegressor(random_state=semilla)
}

# Proceso para cada variable y cada regresor
results = {}
variables = ["NO", "NO2", "SO2", "PM10", "O3", "TMP", "HR", "DD", "PRB", "VV", "RS"]

for var in variables:
    for reg_name, regressor in regressors.items():
        print(f"Procesando {var} con {reg_name}...")
        predictions, mape, rmse = process_variable(
            variable_name=var,
            df=df_concatenado_diario,
            test_size=len(df_test),
            param_grid=param_grid,
            regressor=regressor,
            model_dir=model_directory + reg_name + "_" + tipo_de_df,
            train_mode=False  # Cambiar a False si deseas cargar modelos preentrenados
        )
        results[(var, reg_name)] = {"predictions": predictions, "mape": mape, "rmse": rmse}
        print(f"Resultados para {var} con {reg_name}: MAPE={mape:.2%}, RMSE={rmse:.4f}")

In [ ]:
predictions_df_test = {}

for reg_name in regressors.keys():
    print(f"\nProcesando predicciones para el modelo {reg_name}...\n")
    
    prediction_frames = []
    
    for var in variables:
        predictions_df = results[(var, reg_name)]['predictions']
        
        if 'pred' in predictions_df.columns:
            predictions_df = predictions_df[['pred']]  
            predictions_df.columns = [var] 
        else:
            print(f"Advertencia: No se encontró la columna 'pred' en las predicciones para {var} con {reg_name}.")
            continue
        
        prediction_frames.append(predictions_df)
    
    model_df = pd.concat(prediction_frames, axis=1)
    
    predictions_df_test[reg_name] = model_df
    
    print(f"Predicciones para el modelo {reg_name}:")
    print(model_df.head())  # Mostrar las primeras filas del DataFrame generado


In [ ]:
models = {
    "RandomForest": RandomForestRegressor(random_state=semilla),
    "GradientBoosting": GradientBoostingRegressor(random_state=semilla),
    "XGBoost": XGBRegressor(random_state=semilla)
}

param_grid = {
    'n_estimators': [100, 150, 250],
    'max_depth': [3, 5, 8],
    'min_samples_split': [2, 5, 10]
}

results_models = {}

for model_name, model in models.items():
    print(f"\nRealizando grid search para {model_name}...")

    forecaster = ForecasterRecursive(
        regressor=model,
        lags=7
    )

    cv = TimeSeriesFold(
        steps=7,
        initial_train_size=int(len(y_train) * 0.35),
        refit=True
    )

    results_grid = grid_search_forecaster(
        forecaster=forecaster,
        y=y_train,
        cv=cv,
        param_grid=param_grid,
        metric=[mean_absolute_percent_error, root_mean_squared_error],
        return_best=True,
        n_jobs=-1,
        exog=X_train,  # Utilizar las variables exógenas como X_train_aire
        verbose=True
    )

    best_params = results_grid.loc[results_grid['mean_absolute_percent_error'].idxmin()]
    print(f"Mejores hiperparámetros para {model_name}: {best_params}")
    
    model_params = best_params['params']
    
    # Establecer los parámetros óptimos en el modelo
    model.set_params(**model_params)

    # Ajustar el modelo con los mejores parámetros
    forecaster.regressor = model

    model_filename = "models/" + model_name + "_sin_transformacion/" + "model_NOX_todo.joblib"
    save_forecaster(
            forecaster=forecaster,
            file_name=model_filename,
            save_custom_functions=True,
            verbose=True
        )
    print(f"Modelo guardado como {model_filename}")

    forecaster.fit(y=y_train, exog=X_train)
    
    # Obtener las predicciones con el modelo ajustado
    predicciones = forecaster.predict(steps=len(X_test_aire), exog=predictions_df_test[model_name])

    # Graficar predicciones vs valores reales
    fig, ax = plt.subplots(figsize=(6, 3))
    y_test_aire.plot(ax=ax, label='Real')
    predicciones.plot(ax=ax, label='Predicciones')
    ax.legend()
    plt.title(f'Predicciones con el mejor modelo ({model_name})')
    plt.show()

    # Calcular métricas de error
    mape = mean_absolute_percentage_error(y_test_aire, predicciones)
    rmse = root_mean_squared_error(y_test_aire, predicciones)

    print(f"MAPE para {model_name}: {mape:.2%}")
    print(f"RMSE para {model_name}: {rmse:.4f}")

    results_models[model_name] = {
        "mape": mape,
        "rmse": rmse,
        "predicciones": predicciones
    }

# Mostrar resultados finales de todos los modelos
for model_name, model_results in results_models.items():
    print(f"\nResultados para {model_name}:")
    print(f"MAPE: {model_results['mape']:.2%}")
    print(f"RMSE: {model_results['rmse']:.4f}")


In [ ]:
VV_dft = df_concatenado_diario["VV"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
VV_lagged = create_lagged_features(VV_dft, lags=range(1, 8), horizons=range(1,1), target_column='VV')
VV_lagged.dropna(inplace=True)

X_train = VV_lagged.drop(columns='VV')  # Características (lags)
y_train = VV_lagged['VV']               # Objetivo (VV)

model = load_forecaster('./models/GradientBoosting_sin_transformacion/model_VV.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_VV = model.predict(steps=7)
print(predictions_VV)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
VV_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_VV.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones VV')
plt.show()

In [ ]:
RS_dft = df_concatenado_diario["RS"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
RS_lagged = create_lagged_features(RS_dft, lags=range(1, 8), horizons=range(1,1), target_column='RS')
RS_lagged.dropna(inplace=True)

X_train = RS_lagged.drop(columns='RS')  # Características (lags)
y_train = RS_lagged['RS']               # Objetivo (RS)

model = load_forecaster('./models/GradientBoosting_sin_transformacion/model_RS.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_RS = model.predict(steps=7)
print(predictions_RS)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
RS_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_RS.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones RS')
plt.show()

In [ ]:
predictions_df = pd.DataFrame({
    'NO': predictions_NO,
    'NO2': predictions_NO2,
    'SO2': predictions_NO2,
    'PM10': predictions_PM10,
    'O3': predictions_O3
})

In [ ]:
predictions_df # Visualizamos el nuevo dataset predicho para los siguiente 7 días

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario.drop(columns = ['NOX','TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']

model = load_forecaster('./models/GradientBoosting_sin_transformacion/model_NOX.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_df_concatenado_aire_diario, exog=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = model.predict(steps=7, exog=predictions_df)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

Predicción real: RandomForest

In [ ]:
NO_dft = df_concatenado_diario["NO"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
NO_lagged = create_lagged_features(NO_dft, lags=range(1, 8), horizons=range(1,1), target_column='NO')
NO_lagged.dropna(inplace=True)

X_train = NO_lagged.drop(columns='NO')  # Características (lags)
y_train = NO_lagged['NO']               # Objetivo (NO)

model = load_forecaster('./models/RandomForest_sin_transformacion/model_NO.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_NO = model.predict(steps=7)
print(predictions_NO)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
NO_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_NO.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NO')
plt.show()

In [ ]:
NO2_dft = df_concatenado_diario["NO2"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
NO2_lagged = create_lagged_features(NO2_dft, lags=range(1, 8), horizons=range(1,1), target_column='NO2')
NO2_lagged.dropna(inplace=True)

X_train = NO2_lagged.drop(columns='NO2')  # Características (lags)
y_train = NO2_lagged['NO2']               # Objetivo (NO)

model = load_forecaster('./models/RandomForest_sin_transformacion/model_NO2.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_NO2 = model.predict(steps=7)
print(predictions_NO2)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
NO2_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_NO2.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NO2')
plt.show()

In [ ]:
SO2_dft = df_concatenado_diario["SO2"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
SO2_lagged = create_lagged_features(SO2_dft, lags=range(1, 8), horizons=range(1,1), target_column='SO2')
SO2_lagged.dropna(inplace=True)

X_train = SO2_lagged.drop(columns='SO2')  # Características (lags)
y_train = SO2_lagged['SO2']               # Objetivo (NO)

model = load_forecaster('./models/RandomForest_sin_transformacion/model_SO2.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_SO2 = model.predict(steps=7)
print(predictions_SO2)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
SO2_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_SO2.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones SO2')
plt.show()

In [ ]:
PM10_dft = df_concatenado_diario["PM10"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
PM10_lagged = create_lagged_features(PM10_dft, lags=range(1, 8), horizons=range(1,1), target_column='PM10')
PM10_lagged.dropna(inplace=True)

X_train = PM10_lagged.drop(columns='PM10')  # Características (lags)
y_train = PM10_lagged['PM10']               # Objetivo (NO)

model = load_forecaster('./models/RandomForest_sin_transformacion/model_PM10.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_PM10 = model.predict(steps=7)
print(predictions_PM10)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
PM10_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_PM10.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones PM10')
plt.show()

In [ ]:
O3_dft = df_concatenado_diario["O3"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
O3_lagged = create_lagged_features(O3_dft, lags=range(1, 8), horizons=range(1,1), target_column='O3')
O3_lagged.dropna(inplace=True)

X_train = O3_lagged.drop(columns='O3')  # Características (lags)
y_train = O3_lagged['O3']               # Objetivo (NO)

model = load_forecaster('./models/RandomForest_sin_transformacion/model_O3.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_O3 = model.predict(steps=7)
print(predictions_O3)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
O3_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_O3.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones O3')
plt.show()

In [ ]:
predictions_df = pd.DataFrame({
    'NO': predictions_NO,
    'NO2': predictions_NO2,
    'SO2': predictions_NO2,
    'PM10': predictions_PM10,
    'O3': predictions_O3
})

In [ ]:
predictions_df # Visualizamos el nuevo dataset predicho para los siguiente 7 días

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario.drop(columns = ['NOX','TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']

model = load_forecaster('./models/RandomForest_sin_transformacion/model_NOX.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_df_concatenado_aire_diario, exog=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = model.predict(steps=7, exog=predictions_df)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

Predicción real: XGBoost

In [ ]:
NO_dft = df_concatenado_diario["NO"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
NO_lagged = create_lagged_features(NO_dft, lags=range(1, 8), horizons=range(1,1), target_column='NO')
NO_lagged.dropna(inplace=True)

X_train = NO_lagged.drop(columns='NO')  # Características (lags)
y_train = NO_lagged['NO']               # Objetivo (NO)

model = load_forecaster('./models/XGBoost_sin_transformacion/model_NO.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_NO = model.predict(steps=7)
print(predictions_NO)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
NO_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_NO.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NO')
plt.show()

In [ ]:
NO2_dft = df_concatenado_diario["NO2"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
NO2_lagged = create_lagged_features(NO2_dft, lags=range(1, 8), horizons=range(1,1), target_column='NO2')
NO2_lagged.dropna(inplace=True)

X_train = NO2_lagged.drop(columns='NO2')  # Características (lags)
y_train = NO2_lagged['NO2']               # Objetivo (NO)

model = load_forecaster('./models/XGBoost_sin_transformacion/model_NO2.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_NO2 = model.predict(steps=7)
print(predictions_NO2)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
NO2_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_NO2.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NO2')
plt.show()

In [ ]:
SO2_dft = df_concatenado_diario["SO2"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
SO2_lagged = create_lagged_features(SO2_dft, lags=range(1, 8), horizons=range(1,1), target_column='SO2')
SO2_lagged.dropna(inplace=True)

X_train = SO2_lagged.drop(columns='SO2')  # Características (lags)
y_train = SO2_lagged['SO2']               # Objetivo (NO)

model = load_forecaster('./models/XGBoost_sin_transformacion/model_SO2.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_SO2 = model.predict(steps=7)
print(predictions_SO2)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
SO2_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_SO2.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones SO2')
plt.show()

In [ ]:
PM10_dft = df_concatenado_diario["PM10"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
PM10_lagged = create_lagged_features(PM10_dft, lags=range(1, 8), horizons=range(1,1), target_column='PM10')
PM10_lagged.dropna(inplace=True)

X_train = PM10_lagged.drop(columns='PM10')  # Características (lags)
y_train = PM10_lagged['PM10']               # Objetivo (NO)

model = load_forecaster('./models/XGBoost_sin_transformacion/model_PM10.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_PM10 = model.predict(steps=7)
print(predictions_PM10)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
PM10_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_PM10.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones PM10')
plt.show()

In [ ]:
O3_dft = df_concatenado_diario["O3"].to_frame()  # Convertir la Serie a DataFrame

# Preparar los datos para entrenar el modelo
O3_lagged = create_lagged_features(O3_dft, lags=range(1, 8), horizons=range(1,1), target_column='O3')
O3_lagged.dropna(inplace=True)

X_train = O3_lagged.drop(columns='O3')  # Características (lags)
y_train = O3_lagged['O3']               # Objetivo (NO)

model = load_forecaster('./models/XGBoost_sin_transformacion/model_O3.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_train, exog=None)

# Predicción de los próximos 7 días
predictions_O3 = model.predict(steps=7)
print(predictions_O3)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
O3_dft[-7:].plot(ax=ax, label='Test (real)')
predictions_O3.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones O3')
plt.show()

In [ ]:
predictions_df = pd.DataFrame({
    'NO': predictions_NO,
    'NO2': predictions_NO2,
    'SO2': predictions_NO2,
    'PM10': predictions_PM10,
    'O3': predictions_O3
})

In [ ]:
predictions_df # Visualizamos el nuevo dataset predicho para los siguiente 7 días

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario.drop(columns = ['NOX','TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']

model = load_forecaster('./models/XGBoost_sin_transformacion/model_NOX.joblib') # Cargamos el modelo

# Entrenar el modelo
model.fit(y=y_df_concatenado_aire_diario, exog=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = model.predict(steps=7, exog=predictions_df)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

##### CON TRANSFORMACION

###### Solo variables aire

In [ ]:
porcentaje_entrenamiento = 0.9
df_train_transformed = df_concatenado_diario_transformed[:int(len(df_concatenado_diario_transformed) * porcentaje_entrenamiento)]
df_test_transformed = df_concatenado_diario_transformed[int(len(df_concatenado_diario_transformed) * porcentaje_entrenamiento):]

print(f'Con el porcentaje de %.2f tenemos:' %porcentaje_entrenamiento)
print(f'Tamaño del conjunto de training es %i' %len(df_train))
print(f'Tamaño del conjunto de test es %i' %len(df_test))
print(f'El conjunto de training va de {min(df_train.index)} y {max(df_train.index)}')
print(f'El conjunto de test va de {min(df_test.index)} y {max(df_test.index)}')

In [ ]:
X_train_transformed = df_train_transformed.drop(columns = 'NOX')
y_train_transformed = df_train_transformed['NOX']

X_test_transformed = df_test_transformed.drop(columns='NOX')
y_test_transformed = df_test_transformed['NOX']

In [ ]:
columnas_variables_metereologicas = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS']

X_train_aire_transformed = X_train_transformed.drop(columns=columnas_variables_metereologicas)
y_train_aire_transformed = df_train_transformed['NOX']

X_test_aire_transformed = X_test_transformed.drop(columns=columnas_variables_metereologicas)
y_test_aire_transformed = df_test_transformed['NOX']

In [ ]:
model_directory = "models/"
tipo_de_df = "con_transformacion" #Cambiar con o sin transformación dependiendo

# Hiperparámetros comunes
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15]
}

# Lista de regresores
regressors = {
    "RandomForest": RandomForestRegressor(random_state=semilla),
    "GradientBoosting": GradientBoostingRegressor(random_state=semilla),
    "XGBoost": XGBRegressor(random_state=semilla)
}

# Proceso para cada variable y cada regresor
results = {}
variables = ["NO", "NO2", "SO2", "PM10", "O3"]

for var in variables:
    for reg_name, regressor in regressors.items():
        print(f"Procesando {var} con {reg_name}...")
        predictions, mape, rmse = process_variable(
            variable_name=var,
            df=df_concatenado_aire_diario_transformed,
            test_size=len(df_test),
            param_grid=param_grid,
            regressor=regressor,
            model_dir=model_directory + reg_name + "_" + tipo_de_df,
            train_mode=True  # Cambiar a False si deseas cargar modelos preentrenados
        )
        results[(var, reg_name)] = {"predictions": predictions, "mape": mape, "rmse": rmse}
        print(f"Resultados para {var} con {reg_name}: MAPE={mape:.2%}, RMSE={rmse:.4f}")

In [ ]:
predictions_df_test = {}

for reg_name in regressors.keys():
    print(f"\nProcesando predicciones para el modelo {reg_name}...\n")
    
    prediction_frames = []
    
    for var in variables:
        predictions_df = results[(var, reg_name)]['predictions']
        
        if 'pred' in predictions_df.columns:
            predictions_df = predictions_df[['pred']]  
            predictions_df.columns = [var] 
        else:
            print(f"Advertencia: No se encontró la columna 'pred' en las predicciones para {var} con {reg_name}.")
            continue
        
        prediction_frames.append(predictions_df)
    
    model_df = pd.concat(prediction_frames, axis=1)
    
    predictions_df_test[reg_name] = model_df
    
    print(f"Predicciones para el modelo {reg_name}:")
    print(model_df.head())  # Mostrar las primeras filas del DataFrame generado


In [ ]:
models = {
    "RandomForest": RandomForestRegressor(random_state=semilla),
    "GradientBoosting": GradientBoostingRegressor(random_state=semilla),
    "XGBoost": XGBRegressor(random_state=semilla)
}

param_grid = {
    'n_estimators': [100, 150, 250],
    'max_depth': [3, 5, 8],
    'min_samples_split': [2, 5, 10]
}

results_models = {}

for model_name, model in models.items():
    print(f"\nRealizando grid search para {model_name}...")

    forecaster = ForecasterRecursive(
        regressor=model,
        lags=7
    )

    cv = TimeSeriesFold(
        steps=7,
        initial_train_size=int(len(y_train_aire_transformed) * 0.35),
        refit=True
    )

    results_grid = grid_search_forecaster(
        forecaster=forecaster,
        y=y_train_aire_transformed,
        cv=cv,
        param_grid=param_grid,
        metric=[mean_absolute_percent_error, root_mean_squared_error],
        return_best=True,
        n_jobs=-1,
        exog=X_train_aire_transformed,  # Utilizar las variables exógenas como X_train_aire
        verbose=True
    )

    best_params = results_grid.loc[results_grid['mean_absolute_percent_error'].idxmin()]
    print(f"Mejores hiperparámetros para {model_name}: {best_params}")
    
    model_params = best_params['params']
    
    # Establecer los parámetros óptimos en el modelo
    model.set_params(**model_params)

    # Ajustar el modelo con los mejores parámetros
    forecaster.regressor = model

    model_filename = "models/" + model_name + "_con_transformacion/" + "model_NOX.joblib"
    save_forecaster(
            forecaster=forecaster,
            file_name=model_filename,
            save_custom_functions=True,
            verbose=True
        )
    print(f"Modelo guardado como {model_filename}")

    forecaster.fit(y=y_train_aire_transformed, exog=X_train_aire_transformed)
    
    # Obtener las predicciones con el modelo ajustado
    predicciones = forecaster.predict(steps=len(X_test_aire_transformed), exog=predictions_df_test[model_name])

    # Graficar predicciones vs valores reales
    fig, ax = plt.subplots(figsize=(6, 3))
    y_test_aire.plot(ax=ax, label='Real')
    predicciones.plot(ax=ax, label='Predicciones')
    ax.legend()
    plt.title(f'Predicciones con el mejor modelo ({model_name})')
    plt.show()

    # Calcular métricas de error
    mape = mean_absolute_percentage_error(y_test_aire, predicciones)
    rmse = root_mean_squared_error(y_test_aire, predicciones)

    print(f"MAPE para {model_name}: {mape:.2%}")
    print(f"RMSE para {model_name}: {rmse:.4f}")

    results_models[model_name] = {
        "mape": mape,
        "rmse": rmse,
        "predicciones": predicciones
    }

# Mostrar resultados finales de todos los modelos
for model_name, model_results in results_models.items():
    print(f"\nResultados para {model_name}:")
    print(f"MAPE: {model_results['mape']:.2%}")
    print(f"RMSE: {model_results['rmse']:.4f}")

###### Con todas las variables

In [ ]:
model_directory = "models/"
tipo_de_df = "con_transformacion" #Cambiar con o sin transformación dependiendo

# Hiperparámetros comunes
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15]
}

# Lista de regresores
regressors = {
    "RandomForest": RandomForestRegressor(random_state=semilla),
    "GradientBoosting": GradientBoostingRegressor(random_state=semilla),
    "XGBoost": XGBRegressor(random_state=semilla)
}

# Proceso para cada variable y cada regresor
results = {}
variables = ["TMP", "HR", "DD","PRB", "VV", "RS"]




for var in variables:
    for reg_name, regressor in regressors.items():
        print(f"Procesando {var} con {reg_name}...")
        predictions, mape, rmse = process_variable(
            variable_name=var,
            df=df_concatenado_diario_transformed,
            test_size=len(df_test),
            param_grid=param_grid,
            regressor=regressor,
            model_dir=model_directory + reg_name + "_" + tipo_de_df,
            train_mode=True  # Cambiar a False si deseas cargar modelos preentrenados
        )
        results[(var, reg_name)] = {"predictions": predictions, "mape": mape, "rmse": rmse}
        print(f"Resultados para {var} con {reg_name}: MAPE={mape:.2%}, RMSE={rmse:.4f}")

In [ ]:
model_directory = "models/"
tipo_de_df = "con_transformacion" #Cambiar con o sin transformación dependiendo

# Hiperparámetros comunes
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15]
}

# Lista de regresores
regressors = {
    "RandomForest": RandomForestRegressor(random_state=semilla),
    "GradientBoosting": GradientBoostingRegressor(random_state=semilla),
    "XGBoost": XGBRegressor(random_state=semilla)
}

# Proceso para cada variable y cada regresor
results = {}
variables = ["PRB", "NO", "NO2", "SO2", "PM10", "O3", "TMP", "HR", "DD", "VV", "RS"]
df_concatenado_diario_transformed["PRB"] = df_concatenado_diario["PRB"]
for var in variables:
    for reg_name, regressor in regressors.items():
        print(f"Procesando {var} con {reg_name}...")
        predictions, mape, rmse = process_variable(
            variable_name=var,
            df=df_concatenado_diario_transformed,
            test_size=len(df_test),
            param_grid=param_grid,
            regressor=regressor,
            model_dir=model_directory + reg_name + "_" + tipo_de_df,
            train_mode=False  # Cambiar a False si deseas cargar modelos preentrenados
        )
        results[(var, reg_name)] = {"predictions": predictions, "mape": mape, "rmse": rmse}
        print(f"Resultados para {var} con {reg_name}: MAPE={mape:.2%}, RMSE={rmse:.4f}")

In [ ]:
predictions_df_test = {}

for reg_name in regressors.keys():
    print(f"\nProcesando predicciones para el modelo {reg_name}...\n")
    
    prediction_frames = []
    
    for var in variables:
        predictions_df = results[(var, reg_name)]['predictions']
        
        if 'pred' in predictions_df.columns:
            predictions_df = predictions_df[['pred']]  
            predictions_df.columns = [var] 
        else:
            print(f"Advertencia: No se encontró la columna 'pred' en las predicciones para {var} con {reg_name}.")
            continue
        
        prediction_frames.append(predictions_df)
    
    model_df = pd.concat(prediction_frames, axis=1)
    
    predictions_df_test[reg_name] = model_df
    
    print(f"Predicciones para el modelo {reg_name}:")
    print(model_df.head())  # Mostrar las primeras filas del DataFrame generado

In [ ]:
models = {
    "RandomForest": RandomForestRegressor(random_state=semilla),
    "GradientBoosting": GradientBoostingRegressor(random_state=semilla),
    "XGBoost": XGBRegressor(random_state=semilla)
}

param_grid = {
    'n_estimators': [100, 150, 250],
    'max_depth': [3, 5, 8],
    'min_samples_split': [2, 5, 10]
}

results_models = {}

for model_name, model in models.items():
    print(f"\nRealizando grid search para {model_name}...")

    forecaster = ForecasterRecursive(
        regressor=model,
        lags=7
    )

    cv = TimeSeriesFold(
        steps=7,
        initial_train_size=int(len(y_train) * 0.35),
        refit=True
    )

    X_train_transformed['PRB'] = df_train['PRB']
    X_test_transformed['PRB'] = df_test['PRB']



    results_grid = grid_search_forecaster(
        forecaster=forecaster,
        y=y_train,
        cv=cv,
        param_grid=param_grid,
        metric=[mean_absolute_percent_error, root_mean_squared_error],
        return_best=True,
        n_jobs=-1,
        exog=X_train_transformed,  # Utilizar las variables exógenas como X_train_aire
        verbose=True
    )

    best_params = results_grid.loc[results_grid['mean_absolute_percent_error'].idxmin()]
    print(f"Mejores hiperparámetros para {model_name}: {best_params}")
    
    model_params = best_params['params']
    
    # Establecer los parámetros óptimos en el modelo
    model.set_params(**model_params)

    # Ajustar el modelo con los mejores parámetros
    forecaster.regressor = model

    model_filename = "models/" + model_name + "_con_transformacion/" + "model_NOX_todo.joblib"
    save_forecaster(
            forecaster=forecaster,
            file_name=model_filename,
            save_custom_functions=True,
            verbose=True
        )
    print(f"Modelo guardado como {model_filename}")

    forecaster.fit(y=y_train_transformed, exog=X_train_transformed)
    
    # Obtener las predicciones con el modelo ajustado
    predicciones = forecaster.predict(steps=len(X_test_aire_transformed), exog=predictions_df_test[model_name])

    # Graficar predicciones vs valores reales
    fig, ax = plt.subplots(figsize=(6, 3))
    y_test_aire.plot(ax=ax, label='Real')
    predicciones.plot(ax=ax, label='Predicciones')
    ax.legend()
    plt.title(f'Predicciones con el mejor modelo ({model_name})')
    plt.show()

    # Calcular métricas de error
    mape = mean_absolute_percentage_error(y_test_aire, predicciones)
    rmse = root_mean_squared_error(y_test_aire, predicciones)

    print(f"MAPE para {model_name}: {mape:.2%}")
    print(f"RMSE para {model_name}: {rmse:.4f}")

    results_models[model_name] = {
        "mape": mape,
        "rmse": rmse,
        "predicciones": predicciones
    }

# Mostrar resultados finales de todos los modelos
for model_name, model_results in results_models.items():
    print(f"\nResultados para {model_name}:")
    print(f"MAPE: {model_results['mape']:.2%}")
    print(f"RMSE: {model_results['rmse']:.4f}")

### Predicción directa

### ForecasterDirectMultiVariate

Se va a realizar el ejercicio mediante ForecasterDirectMultiVariate. Esta  estrategia resulta interesante debido a que lo que realiza es una predicción directa  y multivariable. Por tanto, predice los siguientes x días y lo hace teniendo en cuenta todas las series que le introduzcamos. El error dejaría de ser acumulativo y permite manejar las relaciones entre varias variables aunque tiene mayor carga computacional que la predicción recursiva.

Se va a utilizar esta estrategia para cada técnica (GradientBoosting, RandomForest y XGBoost). Por cada uno de ellos se realizan cuatro predicciones:

- **Test con variables de aire:** Busca estudiar las métricas de los modelos de la técnica aplicando solo las variables de aire.
- **Test con variables de aire y meteorológicas:** Busca estudiar las métricas de los modelos de la técnica aplicando todas las variables, es decir, las de aire y las meteorológicas.
- **Predicción real con variables de aire:** Busca predecir realmente los siguientes 7 días aplicando solo las variables de aire.
- **Predicción real con variables de aire y meteorológicas:** Busca predecir realmente los siguientes 7 días aplicando todas las variables, es decir, las de aire y las meteorológicas.

Además, se va a realizar utilizando los datasets sin transformación y con transformación.

#### ForecasterDirectMultiVariate: GradientBoosting sin transformación

##### GradientBoosting test con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_train.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])

forecaster = ForecasterDirectMultiVariate(
                 regressor = GradientBoostingRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_aire[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_aire[:15], predicciones)
rmse = root_mean_squared_error(y_test_aire[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### GradientBoosting test con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_train

forecaster = ForecasterDirectMultiVariate(
                 regressor = GradientBoostingRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_aire[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_aire[:15], predicciones)
rmse = root_mean_squared_error(y_test_aire[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### GradientBoosting predicción real con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = GradientBoostingRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

##### GradientBoosting predicción real con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = GradientBoostingRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

#### ForecasterDirectMultiVariate: RandomForest sin transformación

##### RandomForest test con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_train.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])

forecaster = ForecasterDirectMultiVariate(
                 regressor = RandomForestRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_aire[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_aire[:15], predicciones)
rmse = root_mean_squared_error(y_test_aire[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### RandomForest test con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_train

forecaster = ForecasterDirectMultiVariate(
                 regressor = RandomForestRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_aire[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_aire[:15], predicciones)
rmse = root_mean_squared_error(y_test_aire[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### RandomForest predicción real con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = RandomForestRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

##### RandomForest predicción real con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = RandomForestRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

#### ForecasterDirectMultiVariate: XGBoost sin transformación

##### XGBoost test con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_train.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])

forecaster = ForecasterDirectMultiVariate(
                 regressor = XGBRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_aire[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_aire[:15], predicciones)
rmse = root_mean_squared_error(y_test_aire[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### XGBoost test con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_train

forecaster = ForecasterDirectMultiVariate(
                 regressor = XGBRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_aire[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_aire[:15], predicciones)
rmse = root_mean_squared_error(y_test_aire[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### XGBoost predicción real con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = XGBRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

##### XGBoost predicción real con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario
y_df_concatenado_aire_diario = df_concatenado_diario['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = XGBRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

#### ForecasterDirectMultiVariate: GradientBoosting com transformación

##### GradientBoosting test con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_train_transformed .drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])

forecaster = ForecasterDirectMultiVariate(
                 regressor = GradientBoostingRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_transformed[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_transformed[:15], predicciones)
rmse = root_mean_squared_error(y_test_transformed[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### GradientBoosting test con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_train_transformed

forecaster = ForecasterDirectMultiVariate(
                 regressor = GradientBoostingRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_transformed[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_transformed[:15], predicciones)
rmse = root_mean_squared_error(y_test_transformed[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### GradientBoosting predicción real con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario_transformed.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario_transformed['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = GradientBoostingRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

##### GradientBoosting predicción real con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario_transformed
y_df_concatenado_aire_diario = df_concatenado_diario_transformed['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = GradientBoostingRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

#### ForecasterDirectMultiVariate: RandomForest con transformación

##### RandomForest test con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_train_transformed.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])

forecaster = ForecasterDirectMultiVariate(
                 regressor = RandomForestRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_transformed[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_transformed[:15], predicciones)
rmse = root_mean_squared_error(y_test_transformed[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### RandomForest test con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_train_transformed

forecaster = ForecasterDirectMultiVariate(
                 regressor = RandomForestRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_transformed[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_transformed[:15], predicciones)
rmse = root_mean_squared_error(y_test_transformed[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### RandomForest predicción real con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario_transformed.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario_transformed['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = RandomForestRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

##### RandomForest predicción real con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario_transformed
y_df_concatenado_aire_diario = df_concatenado_diario_transformed['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = RandomForestRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

#### ForecasterDirectMultiVariate: XGBoost con transformación

##### XGBoost test con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_train_transformed.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])

forecaster = ForecasterDirectMultiVariate(
                 regressor = XGBRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_transformed[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_transformed[:15], predicciones)
rmse = root_mean_squared_error(y_test_transformed[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### XGBoost test con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_train_transformed

forecaster = ForecasterDirectMultiVariate(
                 regressor = XGBRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=15,
                 lags= 7

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=15)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_test_transformed[:15].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

# Calcular métricas de error
mape = mean_absolute_percentage_error(y_test_transformed[:15], predicciones)
rmse = root_mean_squared_error(y_test_transformed[:15], predicciones)

print(f"MAPE: {mape:.2%}")
print(f"RMSE: {rmse:.4f}")

##### XGBoost predicción real con variables de aire

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario_transformed.drop(columns = ['TMP', 'HR', 'DD', 'PRB', 'VV', 'RS'])
y_df_concatenado_aire_diario = df_concatenado_diario_transformed['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = XGBRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

##### XGBoost predicción real con variables de aire y meteorológicas

In [ ]:
X_df_concatenado_aire_diario = df_concatenado_diario_transformed
y_df_concatenado_aire_diario = df_concatenado_diario_transformed['NOX']


forecaster = ForecasterDirectMultiVariate(
                 regressor = XGBRegressor(random_state=semilla),
                 level= 'NOX',
                 steps=7,
                 lags= 7,
                 window_features= RollingFeatures(stats=['coef_variation'], window_sizes=[7])

             )

# Entrenar el modelo
forecaster.fit(series=X_df_concatenado_aire_diario)

# Realizar predicciones
predicciones = forecaster.predict(steps=7)

# Imprimir predicciones
print('Predicciones para los próximos días:')
print(predicciones)

# Graficar los datos reales y predicciones
fig, ax = plt.subplots(figsize=(6, 3))
y_df_concatenado_aire_diario[-7:].plot(ax=ax, label='Test (real)')
predicciones.plot(ax=ax, label='Predicciones')
ax.legend()
plt.title('Predicciones NOX')
plt.show()

## Discusión de resultados

**Predicción recursiva**

\begin{array}{|c|c|c|c|c|c|}
\hline
\textbf{Modelo} & \textbf{Transformación} & \textbf{Características} & \textbf{MAPE} & \textbf{RMSE} & \textbf{Hiperparámetros} \\ \hline
RandomForest & Sin & Aire & 13.07\% & 2.0226 & \text{max\_depth=8, min\_samples\_split=5, n\_estimators=250} \\ \hline
GradientBoosting & Sin & Aire & 12.41\% & 2.0091 & \text{max\_depth=5, min\_samples\_split=10, n\_estimators=100} \\ \hline
XGBoost & Sin & Aire & 13.32\% & 2.0930 & \text{max\_depth=5, min\_samples\_split=5, n\_estimators=100} \\ \hline
RandomForest & Sin & Todo & 13.54\% & 2.0691 & \text{max\_depth=8, min\_samples\_split=2, n\_estimators=250} \\ \hline
GradientBoosting & Sin & Todo & 12.57\% & 2.0267 & \text{max\_depth=5, min\_samples\_split=10, n\_estimators=150} \\ \hline
XGBoost & Sin & Todo & 13.30\% & 2.0894 & \text{max\_depth=5, min\_samples\_split=5, n\_estimators=150} \\ \hline
RandomForest & Con & Aire & 11.79\% & 1.9137 & \text{max\_depth=8, min\_samples\_split=5, n\_estimators=250} \\ \hline
GradientBoosting & Con & Aire & 12.19\% & 1.9806 & \text{max\_depth=5, min\_samples\_split=10, n\_estimators=100} \\ \hline
XGBoost & Con & Aire & 12.92\% & 2.0832 & \text{max\_depth=5, min\_samples\_split=5, n\_estimators=100} \\ \hline
\textbf{RandomForest} & Con & Todo & 11.61\% & 1.9123 & \text{max\_depth=8, min\_samples\_split=2, n\_estimators=250} \\ \hline
GradientBoosting & Con & Todo & 12.00\% & 1.9727 & \text{max\_depth=5, min\_samples\_split=10, n\_estimators=150} \\ \hline
XGBoost & Con & Todo & 12.91\% & 2.0796 & \text{max\_depth=5, min\_samples\_split=5, n\_estimators=150} \\ \hline
\end{array}



En el análisis realizado, el modelo **Random Forest** destaca como el mejor en términos de precisión al aplicar una transformación a los datos y usar todas las características disponibles, obteniendo un **MAPE** de 11.61% y un **RMSE** de 1.9123. La transformación de los datos mejora notablemente el desempeño de todos los modelos, siendo más efectiva para Random Forest, que también muestra mejor generalización en diferentes configuraciones. Por otro lado, **Gradient Boosting** es competitivo, aunque generalmente inferior a Random Forest después de la transformación, mientras que **XGBoost** tiene el desempeño más bajo. Usar todas las características en lugar de solo "Aire" también contribuye a una ligera mejora en la precisión general con transformación, sin embargo cuando no hay transformación da mejores resultados solo usando variables relacionadas con la **calidad del aire**.


**Predicción directa**
| Modelo           | Transformación    | Configuración | MAPE    | RMSE   |
|------------------|-------------------|---------------|---------|--------|
| GradientBoosting | Sin Transformación | Con Aire      | 29.14%  | 3.8512 |
| GradientBoosting | Sin Transformación | Con Todo      | 18.77%  | 2.5044 |
| RandomForest     | Sin Transformación | Con Aire      | 4.92%  | 0.6415 |
| RandomForest     | Sin Transformación | Con Todo      | 5.04%   | 0.6472 |
| XGBoost          | Sin Transformación | Con Aire      | 14.93%  | 1.8036 |
| XGBoost          | Sin Transformación | Con Todo      | 10.11%  | 1.2591 |
| GradientBoosting | Con Transformación | Con Aire      | 29.94%  | 3.9333 |
| GradientBoosting | Con Transformación | Con Todo      | 18.86%  | 2.4919 |
| RandomForest     | Con Transformación | Con Aire      | 4.81%   | 0.6347 |
| RandomForest     | Con Transformación | Con Todo      | 4.92%   | 0.6391 |
| XGBoost          | Con Transformación | Con Aire      | 14.93%  | 1.8036 |
| XGBoost          | Con Transformación | Con Todo      | 10.11%  | 1.2591 |



Los resultados de la predicción directa son bastante dispares. Se puede concluir que  la transformación no mejora prácticamente nada, es más, producen prácticamente los mismos resultados.  Destaca también que en GradientBoosting y en XGBoost los modelos con todas las variables dan bastante mejores resultados que los modelos con solo las variables de aire. Ocurre lo contrario con RandomForest, donde solo variables de aire da mejores resultados. Se puede ver que  GradientBoosting da unos resultados bastante malos con 3.8-3.9 de RMSE con solo aire y con un 2.5 de  RMSE con todas las variables. Los mejores modelos son los de RandomForest con un 0.64 de RMSE aproximadamente en todos los modelos. Los modelos de XGBoost no da resultados muy malos, pero en comparación con RandomForest tampoco se pueden considerar que den resultados buenos, su RMSE es de 1.8 con aire y de 1.25 con todo. 

Por tanto, en este caso nos quedaríamos con los modelos de RandomForest, en concreto, nos quedaríamos con el modelo de RandomForest con transformación y con solo las variables de aire, teniendo un MAPE de 4.81% y un RMSE de 0.6347. 

Hay que tener en cuenta que no se ha llegado a realizar grid de hiperparámetros para estos modelos por falta de tiempo y  recursos (el tiempo de entrenamiento era muy elevado). Tal vez si se hubiese configurado un buen grid los resultados de los modelos hubieran mejorado, en especial los de GradientBoosting ya que se ve claramente que sus resultados no son para nada buenos.

----

Una vez comparadas las dos estrategias, nos quedaríamos con el modelo de RandomForest de predicción directa ya que presenta solo un 0.64 de RMSE en comparación con el 1.91 del de recursiva. No obstante, hay que destacar que el modelo recursivo ha predecido la totalidad del conjunto de test, el de predicción directa solo ha predecido los 15 primeros días del conjunto de test. Esto claramente ha podido influir en los resultados y tal vez fuera más conveniente a la larga usar el recursivo si se van a realizar predicciones de muchos días, el modelo directo puede resultarnos útil y dar buenos resultados en predicciones de pocos días, como es nuestro caso donde predecimos los siguientes 7 días. 

## Conclusión

A lo largo de esta práctica, hemos podido observar el comportamiento de distintos modelos para la predicción de series temporales, y más concretamente para la predicción de la variable `NOX`. Entre los modelos estudiados, hemos probado algunos en los que únicamente utilizábamos dicha variable para entrenar el modelo y hacer las predicciones. Estos modelos, al ser más simples, han dado peores resultados. Por un lado, el modelo baseLine, que predecía el valor para un día como el valor del mismo día en el año anterior, ha dado resultados bastante mejorables, lo cual era esperable, debido a su gran simplicidad.

Por otro lado, el modelo de RandomForest con ventana deslizante, si bien ha dado resultados algo mejores que el modelo baseLine, especialmente para el error $RMSE$, seguía teniendo ciertas dificultades para predecir los valores más complejos de nuestro dataset.

A continuación, hemos probado otros modelos en los que ya se podían incluir el resto de variables de nuestro dataset, consiguiendo así una mayor información para realizar las predicciones. Hemos observado que en general, estos modelos han dado mejores resultados que los anteriores modelos en los que únicamente utilizábamos la variable `NOX`. Comenzando por incluir las variables referentes a los datos de calidad del aire, hemos probado un modelo recursivo y otro directo, usando primero los datos originales y luego los transformados, y probando distintas técnicas, y los resultados obtenidos han mejorado considerablemente, especialmente con los modelos recursivos.

Luego, hemos vuelto a usar los mismos modelos, pero esta vez añadiendo además los datos meteorológicos. En este caso, los resultados han empeorado ligeramente para los modelos sin transformación respecto a usar únicamente los datos de calidad del aire, sin embargo para los modelos con transformación ha sido beneficioso usar todas las variables en vez de las relacionadas únicamente con la calidad del aire. Aunque la incorporación de datos adicionales puede, en principio, mejorar las predicciones al aportar más información relevante, en el caso de los modelos sin transformación, la inclusión de las variables meteorológicas no ha producido los resultados esperados. Algunas razones por las que los datos meteorológicos podrían haber empeorado los resultados son:

- **Ruido**: Los datos meteorológicos, aunque pueden ser útiles en ciertas circunstancias, podrían haber introducido ruido en el modelo. Si alguna de las variables meteorológicas no está directamente relacionada con la variable objetivo (`NOX`), estas nuevas características pueden haber confundido al modelo, dificultando la tarea de identificar patrones relevantes y, como consecuencia, aumentando el error.
  
- **Overfitting**: También es posible que la inclusión de más variables haya aumentado la complejidad del modelo, lo que puede haber dado lugar a un mayor overfitting. En ciertos modelos, un mayor número de características puede hacer que el modelo se ajuste demasiado a los datos de entrenamiento, capturando incluso las fluctuaciones aleatorias que no son representativas de las relaciones generales.

En cuanto a los **modelos con transformación**, hemos observado que la mejora en el rendimiento al incluir todas las variables (tanto de calidad del aire como meteorológicas) podría explicarse por varias razones:

1. **Relación no lineal entre variables y respuesta**: Al incluir más variables, algunas de ellas podrían tener una relación no lineal con la variable objetivo (`NOX`). Las transformaciones permiten que los modelos capturen estas relaciones complejas que no se pueden modelar de manera directa.

2. **Escala y unidades de las variables**: Al añadir variables meteorológicas, algunas de ellas pueden tener escalas muy diferentes a las de las variables relacionadas con el aire. Las transformaciones, como la normalización o estandarización, ayudan a igualar la influencia de cada variable en el modelo, evitando que algunas variables dominen sobre otras.

3. **Multicolinealidad**: Al usar todas las variables, es posible que algunas de ellas estén correlacionadas entre sí. Las transformaciones pueden ayudar a reducir la multicolinealidad, facilitando que el modelo se enfoque en las relaciones más importantes sin que las variables redundantes interfieran.

4. **Mejor manejo de la complejidad**: Las transformaciones permiten que modelos más complejos, como los modelos recursivos, puedan ajustarse mejor a la nueva información, capturando patrones y relaciones más detalladas entre las variables.

En resumen, aunque los datos meteorológicos podrían haber mejorado las predicciones al proporcionar información adicional relevante, en este caso parece que su incorporación ha tenido un impacto negativo en el rendimiento de algunos modelos sin transformación. Sin embargo, para los modelos con transformación, el uso de todas las variables ha demostrado ser beneficioso al permitir que las transformaciones manejen mejor las complejidades y relaciones no lineales entre las variables, mejorando así la precisión del modelo. Esto resalta la importancia de la selección y transformación adecuadas de las características, analizando cómo la inclusión de nuevas variables puede mejorar o empeorar el rendimiento del modelo, y cómo las transformaciones pueden ayudar a que los modelos capturen patrones más complejos y no lineales.
